Recommended Top 10 Open Source Art Museums

# Art Museum Country Data License Data Access Recommended Use

1️⃣ The Metropolitan Museum of Art (The Met) 🇺🇸 USA CC0 API

/ CSV Largest, free, high-quality images

2️⃣ Rijksmuseum 🇳🇱 Netherlands CC0 API

Excellent portraits with light and shadow and distinctive clothing features

3️⃣ Tate Britain 🇬🇧 UK CC-BY-NC GitHub CSV

Pre-Raphaelite, Rossetti (key focus)

4️⃣ National Gallery (London) 🇬🇧 UK CC BY API
Core of European classical portraiture

5️⃣ Art Institute of Chicago (AIC) 🇺🇸 USA CC0 API

High-quality portraits, American Impressionism

6️⃣ Cleveland Museum of Art (CMA) 🇺🇸 USA CC0 API
Baroque to Modern
7️⃣ National Gallery of Art (Washington, NGA) 🇺🇸 USA CC0 API
Complete 18th–19th century portrait data
8️⃣ Victoria and Albert Museum (V&A) 🇬🇧 UK Open API
Extensive images of fashion and body posture
9️⃣ Museu Nacional d’Art de Catalunya (MNAC) 🇪🇸 Spain CC BY API
Gothic/Baroque figure data
0️⃣ National Gallery of Australia (NGA-AUS) 🇦🇺 Australia CC BY API / CSV
Expanding non-Eurocentric perspectives

In [ ]:
# main.py
# Embodied Aesthetic Reconstruction — Streamlit App (Resonance-first UI)
# --------------------------------------------------------------------
# - CLIP + Pose + Color + Significance fusion
# - Human-centered UI: semantic "resonance" labels instead of raw numeric scores
# - Sidebar toggle "Debug mode" to show raw similarity when needed
# - Significance score (0–100) + badges + "Learn more"
# - Extra metadata fields: price_estimate_usd, significance_text, interpretive_note_cn
# - iCloud placeholder avoidance, robust pathing, cached embeddings
# - YOLOv8n-pose via HuggingFace (optional), proper device mapping for MPS/CUDA/CPU
# --------------------------------------------------------------------

from __future__ import annotations
import io
import json
import math
import time
from pathlib import Path
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import streamlit as st

# -------------------- Torch / device --------------------
try:
    import torch
    TORCH_OK = True
except Exception:
    TORCH_OK = False

def get_device() -> str:
    """Return 'mps' on Apple Silicon, 'cuda' on Nvidia, else 'cpu'."""
    if not TORCH_OK:
        return "cpu"
    try:
        if torch.backends.mps.is_available():
            return "mps"
        if torch.cuda.is_available():
            return "cuda"
    except Exception:
        pass
    return "cpu"

DEVICE = get_device()
if TORCH_OK:
    try:
        torch.set_float32_matmul_precision("medium")
    except Exception:
        pass

def yolo_device() -> str:
    """Map torch device to Ultralytics' expected device string."""
    if DEVICE == "mps":
        return "mps"
    if DEVICE == "cuda":
        return "0"
    return "cpu"

# -------------------- Paths --------------------
APP_DIR = Path(__file__).parent.resolve()
DATA_DIR = APP_DIR / "data"
IMAGES_DIR = DATA_DIR / "images"
INTERIM_DIR = DATA_DIR / "interim"
CACHE_DIR = APP_DIR / ".clip_cache"
CACHE_DIR.mkdir(exist_ok=True)
INTERIM_DIR.mkdir(parents=True, exist_ok=True)
METADATA_CSV = DATA_DIR / "portrait_works.csv"

# -------------------- UI config --------------------
st.set_page_config(
    page_title="Embodied Aesthetic Reconstruction",
    page_icon="🖼️",
    layout="wide",
)

# -------------------- Utils --------------------
def is_icloud_placeholder(p: Path) -> bool:
    return p.suffix == ".icloud" or p.name.endswith(".icloud")

def load_image_safe(path: Path) -> Optional[Image.Image]:
    if not path.exists() or is_icloud_placeholder(path):
        return None
    try:
        with Image.open(path) as im:
            return im.convert("RGB")
    except Exception:
        return None

def pil_from_bytes(b: bytes) -> Image.Image:
    return Image.open(io.BytesIO(b)).convert("RGB")

def center_crop_long_edge(im: Image.Image, size: int = 512) -> Image.Image:
    im = ImageOps.exif_transpose(im)
    w, h = im.size
    s = min(w, h)
    left = (w - s) // 2
    top = (h - s) // 2
    im = im.crop((left, top, left + s, top + s))
    return im.resize((size, size), Image.BICUBIC)

# -------------------- Sidebar --------------------
with st.sidebar:
    st.header("⚙️ Matching Weights")
    w_clip = st.slider("CLIP weight", 0.0, 1.0, 1.00, 0.05)
    w_pose = st.slider("Pose weight", 0.0, 1.0, 0.30, 0.05)
    w_color = st.slider("Color weight", 0.0, 1.0, 0.20, 0.05)
    w_sig  = st.slider("Significance weight", 0.0, 1.0, 0.20, 0.05)

    st.markdown("---")
    st.header("🔎 Filters")
    require_public = st.checkbox("Require Public-Domain license", value=False)

    st.markdown("---")
    st.header("🧪 Debug")
    show_debug = st.checkbox("Show raw similarity score", value=False)

    st.markdown("---")
    st.header("📊 Results")
    top_k = st.slider("Top-K artworks", 1, 20, 6, 1)

    st.markdown("---")
    st.header("💻 Active device")
    st.success(f"{DEVICE}")

    st.markdown("---")
    st.header("🧍 Overlay")
    overlay_skeleton = st.checkbox("Draw skeleton on preview (if available)", value=True)

# -------------------- Metadata & dataset --------------------
def load_metadata() -> pd.DataFrame:
    """
    Expected (optional) columns:
    filename,title,artist,year,license,museum,accession,movement,
    is_masterwork,citations,exhibitions,auction_price_usd,views_per_year,
    price_estimate_usd,significance_text,interpretive_note_cn,notable_tags,source_links
    """
    if METADATA_CSV.exists():
        try:
            return pd.read_csv(METADATA_CSV)
        except Exception as e:
            st.warning(f"Could not read metadata CSV ({METADATA_CSV.name}): {e}")
    # Fallback minimal schema
    return pd.DataFrame({
        "filename": [], "title": [], "artist": [], "year": [], "license": [],
        "museum": [], "accession": [], "movement": [], "is_masterwork": [],
        "citations": [], "exhibitions": [], "auction_price_usd": [], "views_per_year": [],
        "price_estimate_usd": [], "significance_text": [], "interpretive_note_cn": [],
        "notable_tags": [], "source_links": []
    })

META = load_metadata()

def list_dataset_images() -> List[Path]:
    if not IMAGES_DIR.exists():
        return []
    files: List[Path] = []
    for ext in ("*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"):
        files += list(IMAGES_DIR.glob(ext))
    files = [p for p in files if not is_icloud_placeholder(p)]
    return sorted(files)

DATASET_FILES = list_dataset_images()

# -------------------- Significance (0..100) --------------------
TOP_MUSEUMS = ["met", "national gallery", "tate", "louvre", "uffizi", "hermitage", "ng london"]

def _logn(x, d=1.0):
    try:
        return math.log10(max(float(x), 1.0)) / d
    except Exception:
        return 0.0

def compute_significance_row(row) -> float:
    """Heuristic significance score (0..100)."""
    w = dict(master=0.35, museum=0.20, cites=0.15, exhib=0.10, auction=0.10, views=0.10)
    score = 0.0
    if str(row.get("is_masterwork", "0")).lower() in ["1", "true", "yes", "y"]:
        score += w["master"]
    museum = str(row.get("museum", "")).lower()
    if museum:
        score += w["museum"] * (1.0 if any(m in museum for m in TOP_MUSEUMS) else 0.5)
    score += w["cites"]   * _logn(row.get("citations", 0),        d=3.0)
    score += w["exhib"]   * _logn(row.get("exhibitions", 0),      d=2.0)
    score += w["auction"] * _logn(row.get("auction_price_usd",0), d=8.0)
    score += w["views"]   * _logn(row.get("views_per_year", 0),   d=6.0)
    return float(np.clip(score, 0.0, 1.0) * 100.0)

def significance_badges(row) -> List[str]:
    badges = []
    if str(row.get("is_masterwork","0")).lower() in ["1","true","yes","y"]:
        badges.append("Masterwork")
    if str(row.get("museum","")).strip():
        badges.append("Permanent Collection")
    try:
        if int(row.get("citations", 0)) >= 50:
            badges.append("Canon")
    except Exception:
        pass
    return badges[:3]

def filename_key(path_or_name: str) -> str:
    return Path(path_or_name).name

META_BY_NAME = {}
if not META.empty and "filename" in META.columns:
    for _, r in META.iterrows():
        META_BY_NAME[filename_key(str(r.get("filename","")))] = r

# -------------------- OpenCLIP --------------------
@st.cache_resource(show_spinner=False)
def load_openclip():
    try:
        import open_clip
    except Exception as e:
        st.error(f"open_clip_torch not installed: {e}")
        return None, None, None
    try:
        model_name, pretrained = "ViT-B-32", "laion2b_s34b_b79k"
        model, _, preprocess = open_clip.create_model_and_transforms(
            model_name, pretrained=pretrained, device=DEVICE
        )
        tokenizer = open_clip.get_tokenizer(model_name)
        model.eval()
        return model, preprocess, tokenizer
    except Exception as e:
        st.error(f"Failed to load OpenCLIP: {e}")
        return None, None, None

MODEL_CLIP, PRE_CLIP, TOKENIZER = load_openclip()

def tensor_from_pil_clip(im: Image.Image):
    if PRE_CLIP is None:
        return None
    t = PRE_CLIP(im).unsqueeze(0)
    if TORCH_OK:
        t = t.to(DEVICE)
    return t

@st.cache_data(show_spinner=False)
def embed_image_clip(img_bytes: bytes) -> Optional[np.ndarray]:
    if MODEL_CLIP is None:
        return None
    try:
        im = pil_from_bytes(img_bytes)
        im = center_crop_long_edge(im, size=224)
        x = tensor_from_pil_clip(im)
        with torch.no_grad():
            feat = MODEL_CLIP.encode_image(x)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        return feat.cpu().numpy().astype("float32")[0]
    except Exception:
        return None

# -------------------- YOLO weights via HuggingFace (optional) --------------------
def get_yolo_pose_weights_path() -> Optional[str]:
    try:
        from huggingface_hub import hf_hub_download
    except Exception:
        return "yolov8n-pose.pt"
    try:
        local = hf_hub_download(
            repo_id="ultralytics/yolov8n-pose",
            filename="yolov8n-pose.pt",
            local_dir=str(Path.home() / ".cache" / "hf"),
            local_dir_use_symlinks=False
        )
        return local
    except Exception:
        return "yolov8n-pose.pt"

# -------------------- Feature extraction & cache --------------------
EMB_PATH   = CACHE_DIR / "embeddings.npy"
IDS_PATH   = CACHE_DIR / "ids.json"
POSE_PATH  = CACHE_DIR / "pose.npy"
COLOR_PATH = CACHE_DIR / "color.npy"
SIG_PATH   = CACHE_DIR / "significance.npy"

def _color_feature(im: Image.Image) -> np.ndarray:
    im = im.resize((256, 256), Image.BILINEAR)
    arr = np.array(im.convert("HSV"))
    h, s, v = arr[..., 0], arr[..., 1], arr[..., 2]
    hist_h, _ = np.histogram(h, bins=32, range=(0, 255), density=True)
    hist_s, _ = np.histogram(s, bins=16, range=(0, 255), density=True)
    hist_v, _ = np.histogram(v, bins=8,  range=(0, 255), density=True)
    feat = np.concatenate([hist_h, hist_s, hist_v]).astype("float32")
    return feat / (np.linalg.norm(feat) + 1e-8)

def _pose_feature_from_kpts(kpts_xyv: np.ndarray) -> np.ndarray:
    if kpts_xyv is None or len(kpts_xyv) == 0:
        return np.zeros(34, dtype="float32")
    xy = kpts_xyv[:, :2]
    min_xy = xy.min(0); max_xy = xy.max(0)
    wh = np.maximum(max_xy - min_xy, 1e-6)
    xy_n = (xy - min_xy) / wh
    feat = xy_n.reshape(-1).astype("float32")
    if feat.shape[0] < 34:
        feat = np.pad(feat, (0, 34 - feat.shape[0]))
    else:
        feat = feat[:34]
    return feat / (np.linalg.norm(feat) + 1e-8)

def _detect_pose_pil(im: Image.Image) -> Optional[np.ndarray]:
    try:
        from ultralytics import YOLO
    except Exception:
        return None
    try:
        weights = get_yolo_pose_weights_path()
        model = YOLO(weights)
        arr = np.array(im.convert("RGB"))
        res = model.predict(
            source=arr, imgsz=512, conf=0.25, verbose=False, device=yolo_device()
        )
        if not res or len(res[0].keypoints) == 0:
            return None
        kpts = res[0].keypoints.xy.cpu().numpy()
        if kpts.ndim == 4: kpts = kpts[0]
        if kpts.ndim == 3: k = kpts[0]
        else: return None
        v = np.ones((k.shape[0], 1), dtype="float32")
        kxyv = np.concatenate([k.astype("float32"), v], axis=1)
        return _pose_feature_from_kpts(kxyv)
    except Exception:
        return None

def _dataset_pose_feature(path: Path) -> Optional[np.ndarray]:
    im = load_image_safe(path)
    if im is None:
        return None
    im_small = center_crop_long_edge(im, 512)
    return _detect_pose_pil(im_small)

@st.cache_data(show_spinner=True, persist=True)
def build_dataset_embeddings(files: List[Path]):
    """Return (clip_embeds, ids, pose_feats, color_feats, signif_norm)."""
    ids = [str(p.relative_to(APP_DIR)) for p in files]

    # Use cache if aligned
    if EMB_PATH.exists() and IDS_PATH.exists() and SIG_PATH.exists():
        try:
            cached_ids = json.loads(IDS_PATH.read_text())
            if cached_ids == ids:
                clip_arr  = np.load(EMB_PATH)
                pose_arr  = np.load(POSE_PATH)  if POSE_PATH.exists()  else None
                color_arr = np.load(COLOR_PATH) if COLOR_PATH.exists() else None
                sig_arr   = np.load(SIG_PATH)
                return clip_arr, ids, pose_arr, color_arr, sig_arr
        except Exception:
            pass

    clip_feats, pose_feats, color_feats, signif_vals = [], [], [], []

    if MODEL_CLIP is None:
        st.error("OpenCLIP not ready; cannot build embeddings.")
        return np.zeros((0, 512), dtype="float32"), ids, None, None, np.zeros((0,), dtype="float32")

    progress = st.progress(0.0, text="Building dataset embeddings/features…")
    for i, p in enumerate(files):
        progress.progress((i + 1) / max(1, len(files)))
        im = load_image_safe(p)

        # CLIP
        if im is not None:
            try:
                im_clip = center_crop_long_edge(im, 224)
                x = tensor_from_pil_clip(im_clip)
                with torch.no_grad():
                    z = MODEL_CLIP.encode_image(x)
                    z = z / z.norm(dim=-1, keepdim=True)
                clip_feats.append(z.cpu().numpy().astype("float32")[0])
            except Exception:
                clip_feats.append(np.zeros(512, dtype="float32"))
        else:
            clip_feats.append(np.zeros(512, dtype="float32"))

        # Color
        if im is not None:
            try:
                color_feats.append(_color_feature(im))
            except Exception:
                color_feats.append(np.zeros(56, dtype="float32"))
        else:
            color_feats.append(np.zeros(56, dtype="float32"))

        # Pose
        pf = _dataset_pose_feature(p)
        pose_feats.append(pf if pf is not None else np.zeros(34, dtype="float32"))

        # Significance prior from metadata (0..1)
        fname = filename_key(p.name)
        row = META_BY_NAME.get(fname)
        sig = compute_significance_row(row) / 100.0 if row is not None else 0.0
        signif_vals.append(float(sig))

    clip_arr  = np.vstack(clip_feats).astype("float32") if clip_feats else np.zeros((0, 512), dtype="float32")
    pose_arr  = np.vstack(pose_feats).astype("float32") if pose_feats else None
    color_arr = np.vstack(color_feats).astype("float32") if color_feats else None
    sig_arr   = np.array(signif_vals, dtype="float32")

    # Cache
    try:
        np.save(EMB_PATH, clip_arr)
        np.save(POSE_PATH, pose_arr)
        np.save(COLOR_PATH, color_arr)
        np.save(SIG_PATH,  sig_arr)
        IDS_PATH.write_text(json.dumps(ids))
    except Exception:
        pass

    return clip_arr, ids, pose_arr, color_arr, sig_arr

# ---- Build dataset features ----
if len(DATASET_FILES) == 0:
    st.error(f"❌ No images found or folder is empty: {IMAGES_DIR}\nMake sure iCloud files are downloaded locally (no .icloud).")
else:
    st.success(f"✅ Dataset images: {len(DATASET_FILES)}")

CLIP_DS, IDS, POSE_DS, COLOR_DS, SIG_DS = build_dataset_embeddings(DATASET_FILES)  # SIG_DS in [0,1]

# ---- Ready checks ----
READY = True
reasons = []
if MODEL_CLIP is None:
    READY = False; reasons.append("OpenCLIP not loaded.")
if len(DATASET_FILES) == 0:
    READY = False; reasons.append("No images in data/images (or still .icloud).")
if CLIP_DS is None or getattr(CLIP_DS, "shape", (0,))[0] == 0:
    READY = False; reasons.append("Dataset embeddings not built yet.")
if not READY:
    st.warning("Matching is not ready: " + " ".join(reasons))
else:
    st.success("Matching is ready ✔")

# -------------------- Similarity + Resonance --------------------
def cos_sim(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    if a is None or b is None:
        return None
    if a.ndim == 1:
        a = a[None, :]
    a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-8)
    b = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-8)
    return (a @ b.T)

def interpret_score(score: float) -> str:
    """Map numeric similarity to a poetic/semantic label."""
    if score > 0.80: return "Strong resonance 💫"
    if score > 0.65: return "Aesthetic kinship ✨"
    if score > 0.50: return "Subtle correspondence 🌙"
    return "Distant echo 🌫️"

# -------------------- Pose overlay (preview only) --------------------
def draw_skeleton_overlay(im: Image.Image) -> Tuple[Image.Image, Optional[np.ndarray]]:
    try:
        from ultralytics import YOLO
        import cv2
    except Exception:
        return im, None

    weights = get_yolo_pose_weights_path()
    model = YOLO(weights)
    arr = np.array(im.convert("RGB"))
    res = model.predict(source=arr, imgsz=512, conf=0.25, verbose=False, device=yolo_device())
    if not res or len(res[0].keypoints) == 0:
        return im, None

    canvas = arr.copy()
    pts = res[0].keypoints.xy.cpu().numpy()
    if pts.ndim == 4: pts = pts[0]
    if pts.ndim == 3: pts = pts[0]

    PAIRS = [(5,7),(7,9),(6,8),(8,10),(11,13),(13,15),(12,14),(14,16),(5,6),(11,12),(5,11),(6,12)]
    try:
        import cv2
        for (a,b) in PAIRS:
            xa,ya = pts[a]; xb,yb = pts[b]
            cv2.line(canvas,(int(xa),int(ya)),(int(xb),int(yb)),(0,255,0),3)
        for (x,y) in pts:
            cv2.circle(canvas,(int(x),int(y)),4,(0,0,255),-1)
        im_draw = Image.fromarray(canvas)
    except Exception:
        im_draw = im

    v = np.ones((pts.shape[0],1),dtype="float32")
    kxyv = np.concatenate([pts.astype("float32"), v], axis=1)
    pf = _pose_feature_from_kpts(kxyv)
    return im_draw, pf

# -------------------- UI — Main --------------------
st.title("Embodied Aesthetic Reconstruction")
st.caption("Camera / Upload → CLIP + Pose + Color + Significance → Resonant artworks")

status_cols = st.columns(3)
with status_cols[0]:
    st.success(f"Images dir: {IMAGES_DIR.relative_to(APP_DIR)}" if IMAGES_DIR.exists() else f"Missing: {IMAGES_DIR}")
with status_cols[1]:
    st.info(f"Dataset files: {len(DATASET_FILES)}")
with status_cols[2]:
    st.info(f"Models: OpenCLIP={'✔︎' if MODEL_CLIP else '✖︎'} · Pose(ultralytics)={'✔︎'}")

st.markdown("---")

left, right = st.columns([1, 1])
query_img: Optional[Image.Image] = None
query_pose_feat: Optional[np.ndarray] = None

with left:
    st.subheader("📷 Camera (auto center-crop)")
    cam = st.camera_input("Take a photo (allow permission first)", key="camera")
    if cam is not None:
        try:
            img = pil_from_bytes(cam.getvalue())
            img = center_crop_long_edge(img, 640)
            out_path = INTERIM_DIR / "locked_frame.jpg"
            img.save(out_path, quality=92)
            st.success(f"Saved: {out_path.as_posix()}")
            if overlay_skeleton:
                img_draw, q_pose = draw_skeleton_overlay(img)
                st.image(img_draw, caption="Camera preview", use_container_width=True)
                query_img = img_draw
                query_pose_feat = q_pose
            else:
                st.image(img, caption="Camera preview", use_container_width=True)
                query_img = img
        except Exception as e:
            st.error(f"Camera decode failed: {e}")

with right:
    st.subheader("🖼️ Or upload an image")
    up = st.file_uploader("JPG/PNG", type=["jpg", "jpeg", "png"])
    if up is not None:
        try:
            img = pil_from_bytes(up.getvalue())
            img = center_crop_long_edge(img, 640)
            if overlay_skeleton:
                img_draw, q_pose = draw_skeleton_overlay(img)
                st.image(img_draw, caption="Uploaded preview", use_container_width=True)
                query_img = img_draw
                query_pose_feat = q_pose
            else:
                st.image(img, caption="Uploaded preview", use_container_width=True)
                query_img = img
        except Exception as e:
            st.error(f"Upload decode failed: {e}")

st.markdown("---")
run = st.button("🔎 Run Matching", disabled=not READY)

# -------------------- Display helpers --------------------
def display_results(idx_scores: List[Tuple[int, float]], k: int):
    if len(idx_scores) == 0:
        st.warning("No results to display.")
        return
    cols = st.columns(min(3, k))
    for i, (idx, sc) in enumerate(idx_scores[:k]):
        fn_rel = IDS[idx]
        p = (APP_DIR / fn_rel).resolve()
        im = load_image_safe(p)

        row = META_BY_NAME.get(filename_key(p.name))
        if row is not None:
            title  = str(row.get("title", "")).strip()
            artist = str(row.get("artist", "")).strip()
            year   = str(row.get("year", "")).strip()
            museum = str(row.get("museum", "")).strip()
            price  = str(row.get("price_estimate_usd", "")).strip()
            sigtxt = str(row.get("significance_text", "")).strip()
            sigcn  = str(row.get("interpretive_note_cn", "")).strip()
            links  = str(row.get("source_links", "")).strip()
            sig100 = compute_significance_row(row)
            badges = significance_badges(row)
        else:
            title=artist=year=museum=price=sigtxt=sigcn=links=""
            sig100, badges = 0.0, []

        with cols[i % len(cols)]:
            if im is not None:
                st.image(im, use_container_width=True)

            # Resonance-first display
            if show_debug:
                st.markdown(f"**Score:** {sc:.3f}")
            else:
                st.markdown(f"**{interpret_score(sc)}**")

            if title or artist or year:
                st.caption(f"{title} — {artist} ({year})")
            if museum:
                st.caption(f"🏛️ {museum}")
            if price:
                st.caption(f"💰 Estimated value: ${price}")

            st.markdown(f"**Significance:** {sig100:.0f}/100")
            if badges:
                st.caption(" · ".join(badges))
            if sigtxt:
                st.markdown(f"_{sigtxt}_")
            if sigcn:
                st.markdown(f"**Curatorial note (CN):** {sigcn}")
            if links:
                first = links.split(";")[0].strip()
                if first:
                    st.write(f"[Learn more]({first})")

# -------------------- Matching --------------------
if run:
    if query_img is None:
        st.warning("Please take a photo or upload an image first.")
    else:
        st.write("Computing embeddings…")
        t0 = time.time()

        q_clip = None
        if MODEL_CLIP is not None:
            try:
                buf = io.BytesIO()
                query_img.save(buf, format="JPEG", quality=92)
                q_clip = embed_image_clip(buf.getvalue())
            except Exception:
                q_clip = None

        try:
            q_color = _color_feature(query_img)
        except Exception:
            q_color = None

        if query_pose_feat is None and w_pose > 0:
            query_pose_feat = _detect_pose_pil(center_crop_long_edge(query_img, 512))
        w_pose_eff = w_pose if query_pose_feat is not None and POSE_DS is not None else 0.0
        if w_pose > 0 and w_pose_eff == 0.0:
            st.info("No pose feature detected or dataset pose not precomputed; treating Pose weight as 0.")

        valid_indices = np.arange(len(IDS))
        if require_public and not META.empty and {"filename", "license"}.issubset(META.columns):
            pd_mask = META["license"].astype(str).str.contains("Public Domain", case=False, na=False)
            pool_names = set(META.loc[pd_mask, "filename"].astype(str).apply(lambda s: Path(s).name))
            filt = [i for i, fn in enumerate(IDS) if Path(fn).name in pool_names]
            if len(filt) > 0:
                valid_indices = np.array(filt, dtype=int)
            else:
                st.warning("No entries meet Public-Domain filter; ignoring filter.")

        def slicer(arr):
            if arr is None or arr.shape[0] != len(IDS):
                return None
            return arr[valid_indices]

        CLIP_POOL  = slicer(CLIP_DS)
        POSE_POOL  = slicer(POSE_DS)
        COLOR_POOL = slicer(COLOR_DS)
        SIG_POOL   = SIG_DS[valid_indices] if SIG_DS is not None and SIG_DS.shape[0] == len(IDS) else None

        def cos_sim_local(q, bank):
            if q is None or bank is None:
                return None
            a = q[None, :] if q.ndim == 1 else q
            a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-8)
            b = bank / (np.linalg.norm(bank, axis=1, keepdims=True) + 1e-8)
            return (a @ b.T)[0]

        score = np.zeros(valid_indices.size, dtype="float32")
        s = cos_sim_local(q_clip, CLIP_POOL)
        if s is not None: score += w_clip * s
        s = cos_sim_local(query_pose_feat, POSE_POOL) if w_pose_eff > 0 else None
        if s is not None: score += w_pose_eff * s
        s = cos_sim_local(q_color, COLOR_POOL)
        if s is not None: score += w_color * s
        if SIG_POOL is not None and w_sig > 0.0:
            score += w_sig * SIG_POOL

        topk_local = np.argsort(-score)[:min(top_k, score.size)]
        results = [(int(valid_indices[i]), float(score[i])) for i in topk_local]

        dt = (time.time() - t0) * 1000
        st.info(f"Search time: {dt:.1f} ms")

        display_results(results, k=top_k)

# -------------------- Tips --------------------
with st.expander("Tips"):
    st.markdown(
        """
- If you see **iCloud .icloud** placeholders, open the folder in Finder and choose **“Download Now”**.
- Adjust **CLIP / Pose / Color / Significance** weights to influence ranking.
- **Debug mode** reveals raw similarity scores; keep it off for exhibitions to maintain a poetic tone.
- The **Public-Domain** filter works only if `license` exists in your metadata CSV.
- First run builds and caches dataset features in `.clip_cache/`.
- Pose detection uses `ultralytics` (YOLOv8n-pose). Weights are pulled from Hugging Face when possible.
- *Significance* is a heuristic combining museum presence, citations, and attention. It reflects historical canons yet encourages plural aesthetics.
"""
    )


#frontend

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== 路径与常量 ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# 本地数据（用于根据 filename 找 meta）
DATA_DIR = ROOT_DIR / "data" / "local"
IMAGES_DIR = DATA_DIR / "images"

# 可能存在的 meta CSV
META_CSV_CANDIDATES = [
    DATA_DIR / "portrait_works_enhanced_english.csv",
    DATA_DIR / "portrait_works.csv",
]

# 后端 API（需要时你可以改成 HuggingFace 地址）
DEFAULT_API_URL = "http://127.0.0.1:8000/match"

APP_TITLE = "Embodied Aesthetic Reconstruction"

# YOLOv8-Pose 模型路径（放在 frontend 目录下）
YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

# WebRTC 配置（Safari 需要 STUN）
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness 检测（与策展版一致）
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# 颜色
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

# 展示区域宽度
RIGHT_IMG_MAXW = 900


# =================== 小工具函数 ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """加载本地 CSV，按 filename 建立索引。"""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """根据 filename 在 IMAGES_DIR 里找到图片。"""
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40) -> ImageFont.FreeTypeFont:
    """优先用 Courier / Courier New，找不到再退回 Arial / 默认。"""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """右上角粉色文字：姿态指标（与策展版一致）。"""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    """把关键点变成几行小字（与策展版同样结构）。"""
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """
    在画作上叠加 3 个黄色矩形标签：
    1. 价格（price_text / auction_price_usd）
    2. 年份（year）
    3. 艺术家（artist）
    字体：Courier，黑字，黄色背景，顺序固定。
    """
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    # 从画面中部略靠上开始
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        # 用 textbbox，避免 textsize 报错
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO 视频处理（完全沿用策展骨架风格） ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    与策展版一致：
    - 使用 YOLOv8-Pose 绘制蓝色框 + 绿色骨架
    - 右上角粉色文本显示姿态指标
    - 支持自动静止抓拍 & 手动抓拍
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        """把当前帧与当前指标作为一次抓拍。"""
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]  # YOLO 自带蓝框+绿骨架
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model not available)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== Streamlit 页面（策展布局） ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

# 简单 CSS：两列结构，左摄像头竖屏居中，右图像区域固定宽度
st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}
.block-container {{ padding-top: 0.6rem; padding-bottom: 0.6rem; max-width: 1700px; }}
.left-col .cam-wrap {{
  position: relative;
  height: 92vh;
  width: 100%;
  overflow: hidden;
  border-radius: 12px;
  background: #111;
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 12px !important;
}}
.right-col .art-wrap {{
  position: relative;
  height: 92vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: auto !important;
}}
</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture on demand. "
    "Left: live with pose. Right: matched artwork with tiny pink metrics."
)

# 尝试轻微自动刷新，让自动抓拍时右侧自动更新
try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None

if "last_match" not in st.session_state:
    st.session_state["last_match"] = None  # 保存最后一次 API 返回内容
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0

API_URL = DEFAULT_API_URL  # 如果你希望在 UI 里可编辑，也可以做成 text_input


with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)  # /cam-wrap
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)
    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0
    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet, try again.", icon="✅" if ok else "⚠️")

    # 倒计时逻辑
    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Please hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)  # /left-col


with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        # 从处理器读取最新抓拍
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            # 更新本地时间戳
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            # 把抓拍图发给后端 /match
            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            data = {"museum": "local", "topk": 3}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                payload = resp.json()
                st.session_state["last_match"] = payload
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to trigger matching…")
        elif "error" in payload:
            ph.error(f"Error from backend: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned from backend.")
            else:
                # 只取 Top-1，展陈效果更干净
                top = results[0]
                filename = top.get("filename") or top.get("file")

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image file not found for: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open image: {img_path}")
                    else:
                        # 从本地 CSV 查补充 meta
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist") or top.get("artist") or "artist name",
                            "year": meta_row.get("year") or top.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)

                        # 把抓拍时的姿态指标叠到右上角（粉色文本）
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)  # /art-wrap
    st.markdown("</div>", unsafe_allow_html=True)  # /right-col


main.py

In [ ]:
"""
EAR Backend API (FastAPI + Uvicorn)

Exposes:
- GET  /health     → quick status check
- POST /match      → upload an image and get Top-K matched artworks

It uses the PoseMatcher defined in backend/model/pose_matcher.py and the
Settings from backend/config.py.
"""

from __future__ import annotations

import io
import logging
from typing import List, Optional

from fastapi import FastAPI, File, Form, HTTPException, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from PIL import Image

from .config import Settings
from .model.pose_matcher import PoseMatcher


logger = logging.getLogger("uvicorn.error")

# ----------------------------------------------------------------------
# FastAPI app + CORS
# ----------------------------------------------------------------------

app = FastAPI(
    title="Embodied Aesthetic Reconstruction — Backend API",
    version="0.1.0",
    description="CLIP-based portrait → artwork matching backend.",
)

# Allow everything locally; you can tighten this later if needed.
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global objects filled at startup
settings: Optional[Settings] = None
matcher: Optional[PoseMatcher] = None


# ----------------------------------------------------------------------
# Startup / shutdown
# ----------------------------------------------------------------------


@app.on_event("startup")
def _startup_load() -> None:
    """Load configuration + matcher once when the server starts."""
    global settings, matcher

    logger.info("Loading Settings and PoseMatcher…")
    settings = Settings()
    matcher = PoseMatcher(settings)
    logger.info("Matcher loaded successfully.")


@app.on_event("shutdown")
def _shutdown_cleanup() -> None:
    """Optional cleanup (currently nothing special)."""
    global matcher
    matcher = None
    logger.info("Matcher released.")


# ----------------------------------------------------------------------
# Simple health endpoint
# ----------------------------------------------------------------------


@app.get("/health")
def health():
    return {
        "status": "ok",
        "device": getattr(matcher, "device", "unknown") if matcher else "unloaded",
        "default_museum": getattr(settings, "default_museum", None)
        if settings
        else None,
    }


# ----------------------------------------------------------------------
# Schemas for /match
# ----------------------------------------------------------------------


class MatchResult(BaseModel):
    filename: str
    title: Optional[str] = None
    artist: Optional[str] = None
    year: Optional[int] = None
    score: float


class MatchResponse(BaseModel):
    museum: Optional[str]
    topk: int
    results: List[MatchResult]


# ----------------------------------------------------------------------
# Main matching endpoint
# ----------------------------------------------------------------------


@app.post("/match", response_model=MatchResponse)
async def match(
    image: UploadFile = File(..., description="Uploaded portrait photo"),
    museum: Optional[str] = Form(None, description="Museum name, e.g. local / met"),
    topk: int = Form(3, description="Number of top results"),
):
    """
    Robust matching endpoint:

    - validates that the uploaded file looks like an image
    - decodes it to a PIL image (RGB)
    - calls PoseMatcher.match_pil(...)
    - tries to back-fill missing title / artist / year from matcher.meta_by_filename
    """
    # ----------------- basic validation -----------------
    if topk <= 0:
        topk = 3

    if image.content_type is not None and not image.content_type.startswith("image/"):
        raise HTTPException(
            status_code=400,
            detail=f"Unsupported content type: {image.content_type}. Please upload an image file.",
        )

    data = await image.read()
    if not data:
        raise HTTPException(
            status_code=400,
            detail="Empty file received. Please take a photo again.",
        )

    # ----------------- decode to PIL -----------------
    try:
        pil_img = Image.open(io.BytesIO(data)).convert("RGB")
    except Exception as exc:
        logger.exception("Failed to decode uploaded image")
        raise HTTPException(
            status_code=400,
            detail=f"Invalid image file: {type(exc).__name__}",
        )

    # ----------------- check matcher -----------------
    global matcher
    if matcher is None:
        raise HTTPException(
            status_code=500,
            detail="Matcher is not loaded on the server.",
        )

    effective_museum = museum or getattr(matcher, "default_museum", None) or "local"

    # ----------------- run matcher -----------------
    try:
        if hasattr(matcher, "match_pil"):
            raw_results = matcher.match_pil(
                pil_img,
                museum=effective_museum,
                topk=topk,
            )
        else:
            raise RuntimeError("PoseMatcher has no method 'match_pil'")
    except ValueError as exc:
        logger.exception("ValueError in matcher.match_pil")
        raise HTTPException(
            status_code=400,
            detail=f"Matching failed: {exc}",
        )
    except Exception as exc:
        logger.exception("Unhandled error in matcher.match_pil")
        raise HTTPException(
            status_code=500,
            detail=f"Internal error during matching: {type(exc).__name__}: {exc}",
        )

    # ----------------- normalize results structure -----------------
    payload: List[MatchResult] = []

    meta_store = getattr(matcher, "meta_by_filename", None)

    def fill_from_meta(filename: str, title, artist, year):
        """If title/artist/year are missing, try to fill from matcher metadata."""
        nonlocal meta_store
        if not isinstance(meta_store, dict):
            return title, artist, year

        meta = meta_store.get(filename) or meta_store.get(filename.strip()) or {}
        if isinstance(meta, dict):
            title = title or meta.get("title") or meta.get("Title")
            artist = artist or meta.get("artist") or meta.get("Artist")
            year = year or meta.get("year") or meta.get("Year")
        else:
            title = title or getattr(meta, "title", None)
            artist = artist or getattr(meta, "artist", None)
            year = year or getattr(meta, "year", None)
        return title, artist, year

    for r in raw_results or []:
        if isinstance(r, dict):
            filename = r.get("filename") or r.get("file") or ""
            score = float(r.get("score", 0.0))
            title = r.get("title")
            artist = r.get("artist")
            year = r.get("year")
        else:
            filename = getattr(r, "filename", "") or getattr(r, "file", "")
            score = float(getattr(r, "score", 0.0))
            title = getattr(r, "title", None)
            artist = getattr(r, "artist", None)
            year = getattr(r, "year", None)

        title, artist, year = fill_from_meta(filename, title, artist, year)

        # normalise year → int or None
        year_int: Optional[int]
        try:
            if isinstance(year, str) and year.strip():
                year_int = int(year.split(",")[0].strip())
            elif isinstance(year, (int, float)):
                year_int = int(year)
            else:
                year_int = None
        except Exception:
            year_int = None

        payload.append(
            MatchResult(
                filename=filename,
                title=title,
                artist=artist,
                year=year_int,
                score=float(score),
            )
        )

    return MatchResponse(
        museum=effective_museum,
        topk=topk,
        results=payload,
    )


pose_matcher.py

In [ ]:
"""
PoseMatcher

A lightweight CLIP-based matcher that:

- loads pre-computed embeddings from data/local/embeddings.npy
- loads metadata from data/local/embeddings_meta.csv
- computes an embedding for the query image
- returns the Top-K most similar artworks with their metadata

This version does NOT depend on YOLO pose directly; it only uses
the CLIP image encoder. You already created the embeddings with:

    python backend/tools/build_embeddings.py --museum local ...

So this matcher just consumes those files.
"""

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional

import numpy as np
import open_clip
import torch
from PIL import Image

from backend.config import Settings


@dataclass
class MatchItem:
    filename: str
    title: Optional[str]
    artist: Optional[str]
    year: Optional[int]
    score: float


class PoseMatcher:
    def __init__(self, settings: Settings):
        self.settings = settings

        # --------- Resolve device ----------
        if settings.device == "cpu":
            device = "cpu"
        elif settings.device == "cuda" and torch.cuda.is_available():
            device = "cuda"
        elif settings.device == "mps" and torch.backends.mps.is_available():
            device = "mps"
        else:
            # "auto" or unsupported → try CUDA, then MPS, then CPU
            if torch.cuda.is_available():
                device = "cuda"
            elif torch.backends.mps.is_available():
                device = "mps"
            else:
                device = "cpu"

        self.device = torch.device(device)

        # --------- Load CLIP model ----------
        # It must match the model used in build_embeddings.py
        self.model, self.preprocess, _ = open_clip.create_model_and_transforms(
            "ViT-B-32",
            pretrained="openai",
            device=self.device,
        )
        self.model.eval()

        # --------- Load stored embeddings ----------
        emb_path = settings.embeddings_path
        if not emb_path.exists():
            raise FileNotFoundError(f"Embeddings file not found: {emb_path}")

        # shape: (N, D)
        emb_np = np.load(emb_path)
        if isinstance(emb_np, np.lib.npyio.NpzFile):
            # Just in case someone saved np.savez
            # we expect a "embeddings" key
            emb_np = emb_np["embeddings"]

        # Store as normalized torch tensor on device
        emb = torch.from_numpy(emb_np).float()
        emb = emb / (emb.norm(dim=-1, keepdim=True) + 1e-8)
        self.embeddings = emb.to(self.device)

        # --------- Load metadata ----------
        meta_path = settings.embeddings_meta_path
        if not meta_path.exists():
            raise FileNotFoundError(f"Metadata CSV not found: {meta_path}")

        self.meta: List[Dict] = []
        self.meta_by_filename: Dict[str, Dict] = {}

        with meta_path.open("r", encoding="utf-8") as f:
            header = f.readline().strip().split(",")
            # Simple CSV reader; no quoted commas expected here
            for line in f:
                parts = [p.strip() for p in line.strip().split(",")]
                if not parts or not parts[0]:
                    continue
                row = dict(zip(header, parts))
                filename = row.get("filename") or row.get("file") or ""
                if not filename:
                    continue

                # Normalize year to int if possible
                raw_year = row.get("year") or row.get("Year")
                year_int: Optional[int] = None
                if raw_year:
                    try:
                        year_int = int(str(raw_year).split(",")[0].strip())
                    except Exception:
                        year_int = None

                meta_entry = {
                    "filename": filename,
                    "title": row.get("title") or row.get("Title"),
                    "artist": row.get("artist") or row.get("Artist"),
                    "year": year_int,
                    # Keep raw dict as well in case we need extra columns later
                    **row,
                }
                self.meta.append(meta_entry)
                self.meta_by_filename[filename] = meta_entry

        # For compatibility with the backend/main.py code
        self.default_museum = settings.default_museum

    # ------------------------------------------------------------------
    # Core API used both by backend and Streamlit frontend
    # ------------------------------------------------------------------
    def _encode_image(self, image: Image.Image) -> torch.Tensor:
        """Encode a PIL image into a normalized CLIP embedding."""
        img = self.preprocess(image).unsqueeze(0).to(self.device)  # (1, 3, H, W)
        with torch.no_grad():
            feat = self.model.encode_image(img)  # (1, D)
        feat = feat / (feat.norm(dim=-1, keepdim=True) + 1e-8)
        return feat  # (1, D)

    def match_pil(
        self,
        image: Image.Image,
        museum: Optional[str] = None,
        topk: int = 3,
    ) -> List[Dict]:
        """
        Match a PIL image against the stored embeddings.

        Returns a list of dicts:
            { "filename": ..., "title": ..., "artist": ..., "year": ..., "score": ... }
        """
        if topk <= 0:
            topk = 3

        # Encode query
        query = self._encode_image(image)  # (1, D)

        # Cosine similarity with all stored embeddings
        # Because both are normalized, dot-product == cosine
        sims = (self.embeddings @ query.T).squeeze(1)  # (N,)

        # Top-K indices
        k = min(topk, sims.shape[0])
        scores, indices = torch.topk(sims, k=k, largest=True, sorted=True)

        results: List[Dict] = []
        for score, idx in zip(scores.tolist(), indices.tolist()):
            # index into meta list; we assume same order as embeddings
            if 0 <= idx < len(self.meta):
                meta = self.meta[idx]
            else:
                meta = {}

            filename = meta.get("filename", "")
            title = meta.get("title")
            artist = meta.get("artist")
            year = meta.get("year")

            results.append(
                {
                    "filename": filename,
                    "title": title,
                    "artist": artist,
                    "year": year,
                    "score": float(score),
                }
            )

        return results


utils.py

In [ ]:
# model/utils.py
# -----------------------------------------------------------------------------
# Low-level utilities for image I/O and vector math.
# -----------------------------------------------------------------------------

from __future__ import annotations

import io
from typing import Tuple

import numpy as np
from PIL import Image


def load_image_from_bytes(image_bytes: bytes) -> Image.Image:
    """
    Read an image from raw bytes and convert to RGB PIL Image.
    This function raises if the bytes are not a valid image.
    """
    img = Image.open(io.BytesIO(image_bytes))
    if img.mode != "RGB":
        img = img.convert("RGB")
    return img


def l2_normalize(x: np.ndarray, axis: int = 1, eps: float = 1e-10) -> np.ndarray:
    """
    L2-normalize an array along the given axis.

    Args:
        x: array of shape (..., D) or (N, D) etc.
        axis: axis to normalize across.
        eps: numerical stability term.

    Returns:
        Array with unit L2 norm along `axis`.
    """
    norm = np.linalg.norm(x, ord=2, axis=axis, keepdims=True)
    return x / (norm + eps)


def cosine_similarity(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Compute cosine similarity between rows of `a` and a single vector `b`.

    Assumes both `a` and `b` are already L2-normalized.
    Then cosine similarity reduces to a dot product.

    Args:
        a: (N, D) matrix (database embeddings).
        b: (D,) vector (query embedding).

    Returns:
        (N,) similarity scores.
    """
    # If b has shape (D, 1), squeeze to (D,)
    if b.ndim == 2 and b.shape[1] == 1:
        b = b[:, 0]
    return a @ b  # (N, D) · (D,) -> (N,)


config.py

In [ ]:
"""
Global configuration for the EAR backend.

This file defines a simple Settings model that knows where your
data lives (embeddings, metadata CSV, images directory) and which
device to use for inference.
"""

from pathlib import Path
from pydantic import BaseModel


PROJECT_ROOT = Path(__file__).resolve().parents[1]


class Settings(BaseModel):
    # Root folder that contains embeddings + metadata + images
    data_root: Path = PROJECT_ROOT / "data" / "local"

    # Default museum name used when the frontend does not specify one
    default_museum: str = "local"

    # Device: "auto", "cpu", "cuda", "mps"
    device: str = "auto"

    class Config:
        arbitrary_types_allowed = True

    @property
    def embeddings_path(self) -> Path:
        return self.data_root / "embeddings.npy"

    @property
    def embeddings_meta_path(self) -> Path:
        return self.data_root / "embeddings_meta.csv"

    @property
    def images_dir(self) -> Path:
        return self.data_root / "images"


build_embeddings.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Build CLIP embeddings for a museum folder.

Layout (per museum):
  data/<museum>/
    images/                  # *.jpg / *.png ...
    embeddings.npy           # (N, D)
    embeddings_meta.csv      # at least: filename + optional columns from --meta

Usage:
  python backend/tools/build_embeddings.py --museum local --meta data/local/portrait_works.csv --overwrite
  python backend/tools/build_embeddings.py --all --overwrite

Deps:
  pip install open_clip_torch torch pillow numpy pandas tqdm
"""

import argparse
import os
import sys
from typing import List, Optional

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import open_clip


# ------------------------- utils -------------------------

def detect_device(device_override: Optional[str] = None) -> torch.device:
    if device_override:
        return torch.device(device_override)
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


def list_images(folder: str) -> List[str]:
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tiff"}
    files = []
    for name in sorted(os.listdir(folder)):
        p = os.path.join(folder, name)
        if os.path.isfile(p) and os.path.splitext(name.lower())[1] in exts:
            files.append(p)
    return files


def normalize_rows(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n


def load_external_meta(meta_csv_path: Optional[str]) -> Optional[pd.DataFrame]:
    """Robust CSV reader (BOM-safe, header-trim, filename normalization)."""
    if not meta_csv_path:
        return None
    if not os.path.exists(meta_csv_path):
        print(f"[WARN] metadata CSV not found: {meta_csv_path}")
        return None

    df = pd.read_csv(meta_csv_path, encoding="utf-8-sig")
    df.columns = [c.strip().lower() for c in df.columns]

    # accept 'filename' or common variants
    if "filename" not in df.columns:
        alt = None
        for c in ("image", "file"):
            if c in df.columns:
                alt = c
                break
        if alt:
            df = df.rename(columns={alt: "filename"})

    if "filename" not in df.columns:
        raise ValueError("External metadata CSV must contain a 'filename' (or 'image'/'file') column.")

    df["filename"] = df["filename"].map(lambda x: os.path.basename(str(x)).strip())
    return df


def dual_views(preprocess, pil: Image.Image):
    """full image + center crop (square) as a tiny augmentation ensemble."""
    # full
    im_full = preprocess(pil.convert("RGB"))
    # center crop to min side (square), then preprocess again
    w, h = pil.size
    m = min(w, h)
    left = (w - m) // 2
    top = (h - m) // 2
    pil_square = pil.crop((left, top, left + m, top + m))
    im_square = preprocess(pil_square.convert("RGB"))
    return im_full, im_square


# ------------------------- core -------------------------

def build_for_museum(
    museum: str,
    data_root: str,
    model_name: str = "ViT-B-32",
    pretrained: str = "openai",
    device: Optional[str] = None,
    meta_csv: Optional[str] = None,
    overwrite: bool = False,
    use_dual_view: bool = True,
):
    out_dir = os.path.join(data_root, museum)
    img_dir = os.path.join(out_dir, "images")
    os.makedirs(out_dir, exist_ok=True)

    if not os.path.exists(img_dir):
        print(f"[WARN] images folder not found: {img_dir}")
        return

    files = list_images(img_dir)
    if not files:
        print(f"[WARN] no images found in: {img_dir}")
        return

    dev = detect_device(device)
    print(f"[INFO] museum={museum} | images={len(files)} | device={dev} | model={model_name}/{pretrained}")

    embed_path = os.path.join(out_dir, "embeddings.npy")
    meta_path = os.path.join(out_dir, "embeddings_meta.csv")

    if (os.path.exists(embed_path) or os.path.exists(meta_path)) and not overwrite:
        print(f"[SKIP] embeddings/meta already exist (use --overwrite to rebuild): {out_dir}")
        return

    # model
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=dev)
    model.eval()

    # encode
    feats = []
    for p in tqdm(files, desc=f"[{museum}] progress"):
        try:
            pil = Image.open(p)
        except Exception:
            print(f"[WARN] failed to open image: {p}")
            continue

        if use_dual_view:
            im1, im2 = dual_views(preprocess, pil)
            ims = torch.stack([im1, im2], dim=0).to(dev)
        else:
            ims = torch.stack([preprocess(pil.convert("RGB"))], dim=0).to(dev)

        with torch.no_grad(), torch.autocast(device_type=str(dev), enabled=(dev.type in ["cuda", "mps"])):
            feat = model.encode_image(ims)  # (k, D)
            feat = feat.float().cpu().numpy()
            feat = feat.mean(axis=0, keepdims=True)  # simple average for dual view
        feats.append(feat)

    if not feats:
        print(f"[WARN] no features produced for: {museum}")
        return

    E = np.concatenate(feats, axis=0)  # (N, D)
    E = normalize_rows(E)
    np.save(embed_path, E)
    print(f"[OK] {museum}: embeddings={E.shape} → {embed_path}")

    # base meta (filename only)
    meta_rows = [{"filename": os.path.basename(p)} for p in files]
    base_df = pd.DataFrame(meta_rows)

    # external meta
    ext_df = None
    try:
        ext_df = load_external_meta(meta_csv)
    except Exception as e:
        print(f"[WARN] failed to read external metadata: {e}")

    merged = base_df.merge(ext_df, on="filename", how="left") if ext_df is not None else base_df

    # filename first
    cols = ["filename"] + [c for c in merged.columns if c != "filename"]
    merged = merged[cols]
    merged.to_csv(meta_path, index=False, encoding="utf-8")
    print(f"[OK] {museum}: meta rows={len(merged)} → {meta_path}")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--museum", type=str, help="Museum folder name under data/, e.g., local / met")
    parser.add_argument("--all", action="store_true", help="Build for all subfolders under data/")
    parser.add_argument("--data_root", type=str, default="data")
    parser.add_argument("--model", type=str, default="ViT-B-32")
    parser.add_argument("--pretrained", type=str, default="openai")
    parser.add_argument("--device", type=str, default=None, help="cuda / mps / cpu (auto if omitted)")
    parser.add_argument("--meta", type=str, default=None, help="Optional external CSV to merge (must have filename)")
    parser.add_argument("--overwrite", action="store_true")
    parser.add_argument("--no_dual", action="store_true", help="Disable dual-view averaging")
    args = parser.parse_args()

    if not args.museum and not args.all:
        print("\nPlease set --museum=NAME or use --all.\n")
        sys.exit(1)

    if args.all:
        for m in sorted(d for d in os.listdir(args.data_root)
                        if os.path.isdir(os.path.join(args.data_root, d))):
            build_for_museum(
                museum=m,
                data_root=args.data_root,
                model_name=args.model,
                pretrained=args.pretrained,
                device=args.device,
                meta_csv=args.meta,
                overwrite=args.overwrite,
                use_dual_view=not args.no_dual,
            )
    else:
        build_for_museum(
            museum=args.museum,
            data_root=args.data_root,
            model_name=args.model,
            pretrained=args.pretrained,
            device=args.device,
            meta_csv=args.meta,
            overwrite=args.overwrite,
            use_dual_view=not args.no_dual,
        )


if __name__ == "__main__":
    main()


app_frontend.py

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== 路径与常量 ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# 本地数据（用于根据 filename 找 meta）
DATA_DIR = ROOT_DIR / "data" / "local"
IMAGES_DIR = DATA_DIR / "images"

# 可能存在的 meta CSV
META_CSV_CANDIDATES = [
    DATA_DIR / "portrait_works_enhanced_english.csv",
    DATA_DIR / "portrait_works.csv",
]

# 后端 API（需要时你可以改成 HuggingFace 地址）
DEFAULT_API_URL = "http://127.0.0.1:8000/match"

APP_TITLE = "Embodied Aesthetic Reconstruction"

# YOLOv8-Pose 模型路径（放在 frontend 目录下）
YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

# WebRTC 配置（Safari 需要 STUN）
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness 检测（与策展版一致）
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# 颜色
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

# 展示区域宽度
RIGHT_IMG_MAXW = 900


# =================== 小工具函数 ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """加载本地 CSV，按 filename 建立索引。"""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """根据 filename 在 IMAGES_DIR 里找到图片。"""
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40) -> ImageFont.FreeTypeFont:
    """优先用 Courier / Courier New，找不到再退回 Arial / 默认。"""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """右上角粉色文字：姿态指标（与策展版一致）。"""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    """把关键点变成几行小字（与策展版同样结构）。"""
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """
    在画作上叠加 3 个黄色矩形标签：
    1. 价格（price_text / auction_price_usd）
    2. 年份（year）
    3. 艺术家（artist）
    字体：Courier，黑字，黄色背景，顺序固定。
    """
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    # 从画面中部略靠上开始
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        # 用 textbbox，避免 textsize 报错
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO 视频处理（完全沿用策展骨架风格） ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    与策展版一致：
    - 使用 YOLOv8-Pose 绘制蓝色框 + 绿色骨架
    - 右上角粉色文本显示姿态指标
    - 支持自动静止抓拍 & 手动抓拍
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        """把当前帧与当前指标作为一次抓拍。"""
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]  # YOLO 自带蓝框+绿骨架
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model not available)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== Streamlit 页面（策展布局） ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

# 简单 CSS：两列结构，左摄像头竖屏居中，右图像区域固定宽度
st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}
.block-container {{ padding-top: 0.6rem; padding-bottom: 0.6rem; max-width: 1700px; }}
.left-col .cam-wrap {{
  position: relative;
  height: 92vh;
  width: 100%;
  overflow: hidden;
  border-radius: 12px;
  background: #111;
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 12px !important;
}}
.right-col .art-wrap {{
  position: relative;
  height: 92vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: auto !important;
}}
</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture on demand. "
    "Left: live with pose. Right: matched artwork with tiny pink metrics."
)

# 尝试轻微自动刷新，让自动抓拍时右侧自动更新
try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None

if "last_match" not in st.session_state:
    st.session_state["last_match"] = None  # 保存最后一次 API 返回内容
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0

API_URL = DEFAULT_API_URL  # 如果你希望在 UI 里可编辑，也可以做成 text_input


with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)  # /cam-wrap
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)
    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0
    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet, try again.", icon="✅" if ok else "⚠️")

    # 倒计时逻辑
    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Please hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "No frame yet.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)  # /left-col


with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        # 从处理器读取最新抓拍
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            # 更新本地时间戳
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            # 把抓拍图发给后端 /match
            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            data = {"museum": "local", "topk": 3}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                payload = resp.json()
                st.session_state["last_match"] = payload
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to trigger matching…")
        elif "error" in payload:
            ph.error(f"Error from backend: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned from backend.")
            else:
                # 只取 Top-1，展陈效果更干净
                top = results[0]
                filename = top.get("filename") or top.get("file")

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image file not found for: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open image: {img_path}")
                    else:
                        # 从本地 CSV 查补充 meta
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist") or top.get("artist") or "artist name",
                            "year": meta_row.get("year") or top.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)

                        # 把抓拍时的姿态指标叠到右上角（粉色文本）
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)  # /art-wrap
    st.markdown("</div>", unsafe_allow_html=True)  # /right-col


build_embeddings.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Build CLIP embeddings for a museum folder.

Layout (per museum):
  data/<museum>/
    images/                  # *.jpg / *.png ...
    embeddings.npy           # (N, D)
    embeddings_meta.csv      # at least: filename + optional columns from --meta

Usage:
  python backend/tools/build_embeddings.py --museum local --meta data/local/portrait_works.csv --overwrite
  python backend/tools/build_embeddings.py --all --overwrite

Deps:
  pip install open_clip_torch torch pillow numpy pandas tqdm
"""

import argparse
import os
import sys
from typing import List, Optional

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import open_clip


# ------------------------- utils -------------------------

def detect_device(device_override: Optional[str] = None) -> torch.device:
    if device_override:
        return torch.device(device_override)
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


def list_images(folder: str) -> List[str]:
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tiff"}
    files = []
    for name in sorted(os.listdir(folder)):
        p = os.path.join(folder, name)
        if os.path.isfile(p) and os.path.splitext(name.lower())[1] in exts:
            files.append(p)
    return files


def normalize_rows(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n


def load_external_meta(meta_csv_path: Optional[str]) -> Optional[pd.DataFrame]:
    """Robust CSV reader (BOM-safe, header-trim, filename normalization)."""
    if not meta_csv_path:
        return None
    if not os.path.exists(meta_csv_path):
        print(f"[WARN] metadata CSV not found: {meta_csv_path}")
        return None

    df = pd.read_csv(meta_csv_path, encoding="utf-8-sig")
    df.columns = [c.strip().lower() for c in df.columns]

    # accept 'filename' or common variants
    if "filename" not in df.columns:
        alt = None
        for c in ("image", "file"):
            if c in df.columns:
                alt = c
                break
        if alt:
            df = df.rename(columns={alt: "filename"})

    if "filename" not in df.columns:
        raise ValueError("External metadata CSV must contain a 'filename' (or 'image'/'file') column.")

    df["filename"] = df["filename"].map(lambda x: os.path.basename(str(x)).strip())
    return df


def dual_views(preprocess, pil: Image.Image):
    """full image + center crop (square) as a tiny augmentation ensemble."""
    # full
    im_full = preprocess(pil.convert("RGB"))
    # center crop to min side (square), then preprocess again
    w, h = pil.size
    m = min(w, h)
    left = (w - m) // 2
    top = (h - m) // 2
    pil_square = pil.crop((left, top, left + m, top + m))
    im_square = preprocess(pil_square.convert("RGB"))
    return im_full, im_square


# ------------------------- core -------------------------

def build_for_museum(
    museum: str,
    data_root: str,
    model_name: str = "ViT-B-32",
    pretrained: str = "openai",
    device: Optional[str] = None,
    meta_csv: Optional[str] = None,
    overwrite: bool = False,
    use_dual_view: bool = True,
):
    out_dir = os.path.join(data_root, museum)
    img_dir = os.path.join(out_dir, "images")
    os.makedirs(out_dir, exist_ok=True)

    if not os.path.exists(img_dir):
        print(f"[WARN] images folder not found: {img_dir}")
        return

    files = list_images(img_dir)
    if not files:
        print(f"[WARN] no images found in: {img_dir}")
        return

    dev = detect_device(device)
    print(f"[INFO] museum={museum} | images={len(files)} | device={dev} | model={model_name}/{pretrained}")

    embed_path = os.path.join(out_dir, "embeddings.npy")
    meta_path = os.path.join(out_dir, "embeddings_meta.csv")

    if (os.path.exists(embed_path) or os.path.exists(meta_path)) and not overwrite:
        print(f"[SKIP] embeddings/meta already exist (use --overwrite to rebuild): {out_dir}")
        return

    # model
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=dev)
    model.eval()

    # encode
    feats = []
    for p in tqdm(files, desc=f"[{museum}] progress"):
        try:
            pil = Image.open(p)
        except Exception:
            print(f"[WARN] failed to open image: {p}")
            continue

        if use_dual_view:
            im1, im2 = dual_views(preprocess, pil)
            ims = torch.stack([im1, im2], dim=0).to(dev)
        else:
            ims = torch.stack([preprocess(pil.convert("RGB"))], dim=0).to(dev)

        with torch.no_grad(), torch.autocast(device_type=str(dev), enabled=(dev.type in ["cuda", "mps"])):
            feat = model.encode_image(ims)  # (k, D)
            feat = feat.float().cpu().numpy()
            feat = feat.mean(axis=0, keepdims=True)  # simple average for dual view
        feats.append(feat)

    if not feats:
        print(f"[WARN] no features produced for: {museum}")
        return

    E = np.concatenate(feats, axis=0)  # (N, D)
    E = normalize_rows(E)
    np.save(embed_path, E)
    print(f"[OK] {museum}: embeddings={E.shape} → {embed_path}")

    # base meta (filename only)
    meta_rows = [{"filename": os.path.basename(p)} for p in files]
    base_df = pd.DataFrame(meta_rows)

    # external meta
    ext_df = None
    try:
        ext_df = load_external_meta(meta_csv)
    except Exception as e:
        print(f"[WARN] failed to read external metadata: {e}")

    merged = base_df.merge(ext_df, on="filename", how="left") if ext_df is not None else base_df

    # filename first
    cols = ["filename"] + [c for c in merged.columns if c != "filename"]
    merged = merged[cols]
    merged.to_csv(meta_path, index=False, encoding="utf-8")
    print(f"[OK] {museum}: meta rows={len(merged)} → {meta_path}")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--museum", type=str, help="Museum folder name under data/, e.g., local / met")
    parser.add_argument("--all", action="store_true", help="Build for all subfolders under data/")
    parser.add_argument("--data_root", type=str, default="data")
    parser.add_argument("--model", type=str, default="ViT-B-32")
    parser.add_argument("--pretrained", type=str, default="openai")
    parser.add_argument("--device", type=str, default=None, help="cuda / mps / cpu (auto if omitted)")
    parser.add_argument("--meta", type=str, default=None, help="Optional external CSV to merge (must have filename)")
    parser.add_argument("--overwrite", action="store_true")
    parser.add_argument("--no_dual", action="store_true", help="Disable dual-view averaging")
    args = parser.parse_args()

    if not args.museum and not args.all:
        print("\nPlease set --museum=NAME or use --all.\n")
        sys.exit(1)

    if args.all:
        for m in sorted(d for d in os.listdir(args.data_root)
                        if os.path.isdir(os.path.join(args.data_root, d))):
            build_for_museum(
                museum=m,
                data_root=args.data_root,
                model_name=args.model,
                pretrained=args.pretrained,
                device=args.device,
                meta_csv=args.meta,
                overwrite=args.overwrite,
                use_dual_view=not args.no_dual,
            )
    else:
        build_for_museum(
            museum=args.museum,
            data_root=args.data_root,
            model_name=args.model,
            pretrained=args.pretrained,
            device=args.device,
            meta_csv=args.meta,
            overwrite=args.overwrite,
            use_dual_view=not args.no_dual,
        )


if __name__ == "__main__":
    main()


build_embeddings.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Build pose embeddings for a museum folder.

For now this uses the same CLIP encoder as build_embeddings.py,
so everything works out-of-the-box. Later you can replace it with a
true pose encoder (e.g. from YOLO keypoints).

Layout (per museum):
  data/<museum>/
    images/
    pose_embeddings.npy      # (N, D_pose)  (same N as embeddings.npy)

Usage example:
  python backend/tools/build_pose_embeddings.py --museum mixed --data_root data

Deps:
  pip install open_clip_torch torch pillow numpy tqdm
"""

import argparse
import os
from typing import List, Optional

import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import open_clip


def detect_device(device_override: Optional[str] = None) -> torch.device:
    if device_override:
        return torch.device(device_override)
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


def list_images(folder: str) -> List[str]:
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tiff"}
    files = []
    for name in sorted(os.listdir(folder)):
        p = os.path.join(folder, name)
        if os.path.isfile(p) and os.path.splitext(name.lower())[1] in exts:
            files.append(p)
    return files


def normalize_rows(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n


def build_pose_for_museum(
    museum: str,
    data_root: str = "data",
    model_name: str = "ViT-B-32",
    pretrained: str = "openai",
    device: Optional[str] = None,
    overwrite: bool = False,
):
    base = os.path.join(data_root, museum)
    img_dir = os.path.join(base, "images")
    pose_path = os.path.join(base, "pose_embeddings.npy")

    if not os.path.isdir(img_dir):
        print(f"[WARN] images folder not found: {img_dir}")
        return

    if os.path.exists(pose_path) and not overwrite:
        print(f"[SKIP] {museum}: pose_embeddings already exists ({pose_path})")
        return

    files = list_images(img_dir)
    if not files:
        print(f"[WARN] no images in {img_dir}")
        return

    dev = detect_device(device)
    print(f"[INFO] building pose embeddings for {museum} on {dev}…")

    model, preprocess, _ = open_clip.create_model_and_transforms(
        model_name,
        pretrained=pretrained,
        device=dev,
    )
    model.eval()

    feats = []
    for path in tqdm(files, desc=f"{museum}"):
        try:
            pil = Image.open(path).convert("RGB")
        except Exception as e:
            print(f"[WARN] failed to open {path}: {e}")
            continue

        im = preprocess(pil).unsqueeze(0).to(dev)
        with torch.no_grad(), torch.autocast(device_type=str(dev), enabled=(dev.type in ["cuda", "mps"])):
            feat = model.encode_image(im)
            feat = feat.float().cpu().numpy()
        feats.append(feat)

    if not feats:
        print(f"[WARN] no features produced for: {museum}")
        return

    E = np.concatenate(feats, axis=0)  # (N, D)
    E = normalize_rows(E)
    np.save(pose_path, E)
    print(f"[OK] {museum}: pose_embeddings={E.shape} → {pose_path}")


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--museum", type=str, help="Museum folder name under data/, e.g. local / met / mixed")
    parser.add_argument("--data_root", type=str, default="data")
    parser.add_argument("--model", type=str, default="ViT-B-32")
    parser.add_argument("--pretrained", type=str, default="openai")
    parser.add_argument("--device", type=str, default=None)
    parser.add_argument("--overwrite", action="store_true")
    args = parser.parse_args()

    if not args.museum:
        print("\nPlease set --museum=NAME (e.g. --museum=mixed).\n")
        raise SystemExit(1)

    build_pose_for_museum(
        museum=args.museum,
        data_root=args.data_root,
        model_name=args.model,
        pretrained=args.pretrained,
        device=args.device,
        overwrite=args.overwrite,
    )


if __name__ == "__main__":
    main()


config.py

In [ ]:
"""
Global configuration for the EAR backend.

This file defines a simple Settings model that knows where your
data lives (embeddings, metadata CSV, images directory) and which
device to use for inference.
"""

from pathlib import Path
from pydantic import BaseModel


PROJECT_ROOT = Path(__file__).resolve().parents[1]


PROJECT_ROOT = Path(__file__).resolve().parents[1]


class Settings(BaseModel):
    # 使用混合索引：local + met
    data_root: Path = PROJECT_ROOT / "data" / "mixed"

    # 默认“馆”名随便写，但统一一下
    default_museum: str = "mixed"

    # Device: "auto", "cpu", "cuda", "mps"
    device: str = "auto"


    # Device: "auto", "cpu", "cuda", "mps"
    device: str = "auto"

    class Config:
        arbitrary_types_allowed = True

    @property
    def embeddings_path(self) -> Path:
        return self.data_root / "embeddings.npy"

    @property
    def embeddings_meta_path(self) -> Path:
        return self.data_root / "embeddings_meta.csv"

    @property
    def images_dir(self) -> Path:
        return self.data_root / "images"

app_frontend.py (in Chinese)

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== 路径与常量 ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# 使用 global mixed 数据（local + met 合并后的全局索引）
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# 可能存在的 meta CSV：优先 mixed 的 embeddings_meta.csv，再回退到 local 的 csv
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

# 后端 API（需要时你可以改成 HuggingFace 地址）
DEFAULT_API_URL = "http://127.0.0.1:8000/match"

APP_TITLE = "Embodied Aesthetic Reconstruction"

# YOLOv8-Pose 模型路径（放在 frontend 目录下）
YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

# WebRTC 配置（Safari 需要 STUN）
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness 检测（与策展版一致）
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# 颜色
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

# 展示区域宽度
RIGHT_IMG_MAXW = 900


# =================== 小工具函数 ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """加载 CSV，按 filename 建立索引。"""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """根据 filename 在 IMAGES_DIR 里找到图片。"""
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40) -> ImageFont.FreeTypeFont:
    """优先用 Courier / Courier New，找不到再退回 Arial / 默认。"""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """右上角粉色文字：姿态指标（与策展版一致）。"""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    """把关键点变成几行小字（与策展版同样结构）。"""
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """
    在画作上叠加 3 个黄色矩形标签：
    1. 价格（price_text / auction_price_usd）
    2. 年份（year）
    3. 艺术家（artist）
    字体：Courier，黑字，黄色背景，顺序固定。
    """
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    # 从画面中部略靠上开始
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """兼容新旧 Streamlit 的 rerun。"""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO 视频处理（沿用策展骨架风格） ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    与策展版一致：
    - 使用 YOLOv8-Pose 绘制蓝色框 + 绿色骨架
    - 右上角粉色文本显示姿态指标
    - 支持自动静止抓拍 & 手动抓拍
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        """把当前帧与当前指标作为一次抓拍。"""
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]  # YOLO 自带蓝框+绿骨架
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model not available)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== Streamlit 页面（策展布局） ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

# 简单 CSS：两列结构，左摄像头竖屏居中，右图像区域固定宽度
st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}
.block-container {{ padding-top: 0.6rem; padding-bottom: 0.6rem; max-width: 1700px; }}
.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 12px;
  background: #111;
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 12px !important;
}}
.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}
</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture on demand. "
    "Left: live with pose. Right: matched artwork with tiny pink metrics."
)

# 轻微自动刷新，让自动抓拍时右侧自动更新
try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None

if "last_match" not in st.session_state:
    st.session_state["last_match"] = None  # 保存最后一次 API 返回内容
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0

API_URL = DEFAULT_API_URL  # 如果你希望在 UI 里可编辑，也可以做成 text_input


with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)  # /cam-wrap
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)
    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0
    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast(
                    "Captured." if ok else "No frame yet, try again.",
                    icon="✅" if ok else "⚠️",
                )

    # 倒计时逻辑
    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Please hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast(
                    "Captured." if ok else "No frame yet.",
                    icon="✅" if ok else "⚠️",
                )
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)  # /left-col


with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        # 从处理器读取最新抓拍
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            # 更新本地时间戳
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            # 把抓拍图发给后端 /match
            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # 使用 global mixed index，一次取 Top-3，但只展示 Top-1
            data = {"museum": "mixed", "topk": 3}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                payload = resp.json()
                st.session_state["last_match"] = payload
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to trigger matching…")
        elif "error" in payload:
            ph.error(f"Error from backend: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned from backend.")
            else:
                # 只取 Top-1，展陈效果更干净
                top = results[0]
                filename = top.get("filename") or top.get("file")

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image file not found for: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open image: {img_path}")
                    else:
                        # 从 CSV 查补充 meta
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or top.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or top.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)

                        # 把抓拍时的姿态指标叠到右上角（粉色文本）
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)  # /art-wrap
    st.markdown("</div>", unsafe_allow_html=True)  # /right-col


app_fronted.py(in English)

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# Meta CSV candidates (fallback to local)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

# Backend API
DEFAULT_API_URL = "http://127.0.0.1:8000/match"

APP_TITLE = "Embodied Aesthetic Reconstruction"

# YOLO model path
YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

# WebRTC ICE config (Safari requires STUN)
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness & motion detection
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# Colors
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(im: Image.Image, lines: List[str], size=16, margin=10):
    """Small pink pose metrics in top-right corner of image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin

    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)

    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """
    Exhibition-style yellow labels:
    1. Price
    2. Year
    3. Artist
    """
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + pad_x * 2, h + pad_y * 2

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old versions of Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO
    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto-capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb):
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        return tuple(b[i].astype(int).tolist())

    def _extract_keypoints(self, res):
        kps = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / (w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))

        stable = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)

            with self.lock:
                self.last_metrics_lines = lines

            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            out_rgb = img_rgb
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}
.block-container {{ padding-top: 0.6rem; padding-bottom: 0.6rem; max-width: 1700px; }}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;               /* was 92vh */
  width: 100%;
  overflow: hidden;
  border-radius: 12px;
  background: #111;
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 12px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;               /* was 92vh */
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}
</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink metrics."
)

try:
    st.autorefresh(interval=700, key="auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            data = {"museum": "mixed", "topk": 3}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                top = results[0]
                filename = top.get("filename") or top.get("file")

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or top.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or top.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)

                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(painted, metrics, size=16)

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


679

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
import random
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# Meta CSV candidates (fallback to local if needed)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

DEFAULT_API_URL = "http://127.0.0.1:8000/match"
APP_TITLE = "Embodied Aesthetic Reconstruction"

YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900

# Diversity / randomness control
MAX_RECENT = 6          # how many artworks to remember
PRIMARY_TOP_K = 3       # "very best" range
TAIL_TOP_K = 10         # search in top-10 in total
TAIL_RANDOM_PROB = 0.3  # 30% chance to look into 4–10 first


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """Small pink pose metrics in top-right corner of the image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """Yellow label stack: price, year, artist."""
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}

body {{
  background-color: #f5f5f7;
}}
.block-container {{
  padding-top: 0.6rem;
  padding-bottom: 0.6rem;
  max-width: 1700px;
}}

h1 {{
  font-family: -apple-system, BlinkMacSystemFont, "SF Pro Display", system-ui, sans-serif;
  letter-spacing: 0.04em;
  font-weight: 700;
}}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 18px;
  background: #111;
  box-shadow: 0 18px 45px rgba(0,0,0,0.3);
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 18px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
  border-radius: 18px;
  background: #050505;
  box-shadow: 0 18px 45px rgba(0,0,0,0.35);
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}

button[kind="secondary"] {{
  border-radius: 999px !important;
}}

</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink pose metrics."
)

try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0
if "recent_files" not in st.session_state:
    st.session_state["recent_files"] = []  # type: List[str]

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # Ask backend for Top-10, we'll do diversity selection here
            data = {"museum": "mixed", "topk": TAIL_TOP_K}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                recent = st.session_state.get("recent_files", [])

                # ---------- diversity + mild randomness ----------
                # Decide whether to start looking from the tail (ranks 4–10)
                use_tail_first = random.random() < TAIL_RANDOM_PROB and len(results) > PRIMARY_TOP_K
                primary_slice = results[:PRIMARY_TOP_K]
                tail_slice = results[PRIMARY_TOP_K:TAIL_TOP_K]

                ordered_candidates = (
                    (tail_slice + primary_slice) if use_tail_first else (primary_slice + tail_slice)
                )

                chosen = None
                chosen_fname = None

                for r in ordered_candidates:
                    fname = (
                        r.get("filename")
                        or r.get("file")
                        or r.get("image_path")
                        or r.get("path")
                    )
                    if not fname:
                        continue
                    if fname not in recent:
                        chosen = r
                        chosen_fname = fname
                        break

                # Fallback: if all candidates are "recent", use strict Top-1
                if chosen is None:
                    chosen = results[0]
                    chosen_fname = (
                        chosen.get("filename")
                        or chosen.get("file")
                        or chosen.get("image_path")
                        or chosen.get("path")
                    )

                # Update recent memory
                if chosen_fname:
                    if chosen_fname in recent:
                        recent.remove(chosen_fname)
                    recent.insert(0, chosen_fname)
                    del recent[MAX_RECENT:]
                    st.session_state["recent_files"] = recent

                filename = chosen_fname

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or chosen.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or chosen.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(painted, metrics, size=16, margin=12)

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


pose_matcher.py

In [ ]:
"""
PoseMatcher

CLIP-based matcher with optional *pose embeddings* re-weighting:

- loads CLIP embeddings from data/.../embeddings.npy
- loads optional pose embeddings from data/.../pose_embeddings.npy
- loads metadata from data/.../embeddings_meta.csv
- computes query CLIP (and pose) embeddings for the uploaded image
- mixes scores: (1 - pose_weight) * clip + pose_weight * pose
- returns Top-K matched artworks with their metadata
"""

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional

import numpy as np
import open_clip
import torch
from PIL import Image

from backend.config import Settings


@dataclass
class MatchItem:
    filename: str
    title: Optional[str]
    artist: Optional[str]
    year: Optional[int]
    score: float


class PoseMatcher:
    def __init__(self, settings: Settings):
        self.settings = settings

        # --------- Resolve device ----------
        if settings.device == "cpu":
            device = "cpu"
        elif settings.device == "cuda" and torch.cuda.is_available():
            device = "cuda"
        elif settings.device == "mps" and torch.backends.mps.is_available():
            device = "mps"
        else:
            # "auto" or unsupported → try CUDA, then MPS, then CPU
            if torch.cuda.is_available():
                device = "cuda"
            elif torch.backends.mps.is_available():
                device = "mps"
            else:
                device = "cpu"
        self.device = torch.device(device)

        # --------- Load CLIP model ----------
        # Must match build_embeddings.py
        self.model, self.preprocess, _ = open_clip.create_model_and_transforms(
            "ViT-B-32",
            pretrained="openai",
            device=self.device,
        )
        self.model.eval()

        # --------- Load CLIP embeddings ----------
        emb_path = settings.embeddings_path
        if not emb_path.exists():
            raise FileNotFoundError(f"Embeddings file not found: {emb_path}")

        emb_np = np.load(emb_path)
        if isinstance(emb_np, np.lib.npyio.NpzFile):
            emb_np = emb_np["embeddings"]

        emb = torch.from_numpy(emb_np).float()
        emb = emb / (emb.norm(dim=-1, keepdim=True) + 1e-8)
        self.embeddings = emb.to(self.device)  # (N, D_clip)

        # --------- Load pose embeddings (optional) ----------
        self.pose_embeddings: Optional[torch.Tensor] = None
        self.pose_weight: float = float(settings.pose_weight)

        pose_path: Path = settings.pose_embeddings_path
        if pose_path.exists():
            pose_np = np.load(pose_path)
            # basic sanity check: same number of rows as CLIP embeddings
            if pose_np.shape[0] == emb_np.shape[0]:
                pose = torch.from_numpy(pose_np).float()
                pose = pose / (pose.norm(dim=-1, keepdim=True) + 1e-8)
                self.pose_embeddings = pose.to(self.device)  # (N, D_pose)
            else:
                print(
                    f"[WARN] pose_embeddings shape {pose_np.shape} "
                    f"does not match embeddings {emb_np.shape}; ignoring pose features."
                )
        else:
            # Not an error: we simply fall back to CLIP-only
            print(f"[INFO] pose_embeddings not found at {pose_path}, using CLIP-only scores.")

        # --------- Load metadata ----------
        meta_path = settings.embeddings_meta_path
        if not meta_path.exists():
            raise FileNotFoundError(f"Metadata CSV not found: {meta_path}")

        self.meta: List[Dict] = []
        self.meta_by_filename: Dict[str, Dict] = {}

        with meta_path.open("r", encoding="utf-8") as f:
            header = f.readline().strip().split(",")
            for line in f:
                parts = [p.strip() for p in line.strip().split(",")]
                if not parts or not parts[0]:
                    continue
                row = dict(zip(header, parts))
                filename = row.get("filename") or row.get("file") or ""
                if not filename:
                    continue

                # Normalize year to int if possible
                raw_year = row.get("year") or row.get("Year")
                year_int: Optional[int] = None
                if raw_year:
                    try:
                        year_int = int(str(raw_year).split(",")[0].strip())
                    except Exception:
                        year_int = None

                meta_entry = {
                    "filename": filename,
                    "title": row.get("title") or row.get("Title"),
                    "artist": row.get("artist") or row.get("Artist"),
                    "year": year_int,
                    # keep all raw columns in case we need them later
                    **row,
                }
                self.meta.append(meta_entry)
                self.meta_by_filename[filename] = meta_entry

        # For compatibility with backend.main
        self.default_museum = settings.default_museum

    # ------------------------------------------------------------------
    # Encoding helpers
    # ------------------------------------------------------------------
    def _encode_image(self, image: Image.Image) -> torch.Tensor:
        """
        Encode a PIL image into a normalized CLIP embedding (1, D_clip).
        """
        img = self.preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            feat = self.model.encode_image(img)
        feat = feat / (feat.norm(dim=-1, keepdim=True) + 1e-8)
        return feat  # (1, D_clip)

    def _encode_pose(self, image: Image.Image) -> torch.Tensor:
        """
        Encode pose-style features for the image.

        For now this is implemented as the same CLIP encoder
        (so that everything runs out-of-the-box). Later you can
        replace this with a dedicated pose encoder or a vector
        built from YOLO keypoints.

        Shape: (1, D_pose) — must match pose_embeddings' last dim.
        """
        # Placeholder implementation = CLIP features again
        return self._encode_image(image)

    # ------------------------------------------------------------------
    # Public APIs
    # ------------------------------------------------------------------
    def match_pil(
        self,
        image: Image.Image,
        museum: Optional[str] = None,
        topk: int = 3,
    ) -> List[Dict]:
        """
        Match a PIL image against the stored embeddings.

        Returns a list of dicts:
            { "filename": ..., "title": ..., "artist": ..., "year": ..., "score": ... }
        """
        if topk <= 0:
            topk = 3

        # ---- CLIP similarity ----
        q_clip = self._encode_image(image)  # (1, D_clip)
        sims_clip = (self.embeddings @ q_clip.T).squeeze(1)  # (N,)

        # ---- pose similarity (optional) ----
        if self.pose_embeddings is not None and self.pose_weight > 0.0:
            q_pose = self._encode_pose(image)  # (1, D_pose)
            # if dims mismatch, fall back to CLIP-only
            if q_pose.shape[1] == self.pose_embeddings.shape[1]:
                sims_pose = (self.pose_embeddings @ q_pose.T).squeeze(1)
                alpha = float(self.pose_weight)
                sims = (1.0 - alpha) * sims_clip + alpha * sims_pose
            else:
                print(
                    f"[WARN] pose dim mismatch: query={q_pose.shape}, "
                    f"embeddings={self.pose_embeddings.shape}; using CLIP-only."
                )
                sims = sims_clip
        else:
            sims = sims_clip

        # Top-K indices
        k = min(topk, sims.shape[0])
        scores, indices = torch.topk(sims, k=k, largest=True, sorted=True)

        results: List[Dict] = []
        for score, idx in zip(scores.tolist(), indices.tolist()):
            if 0 <= idx < len(self.meta):
                meta = self.meta[idx]
            else:
                meta = {}

            filename = meta.get("filename", "")
            title = meta.get("title")
            artist = meta.get("artist")
            year = meta.get("year")

            results.append(
                {
                    "filename": filename,
                    "title": title,
                    "artist": artist,
                    "year": year,
                    "score": float(score),
                }
            )
        return results

    def match_image_bytes(
        self,
        image_bytes: bytes,
        museum: Optional[str] = None,
        topk: int = 3,
    ) -> List[Dict]:
        """
        Convenience wrapper used by the purely local Streamlit curatorial app.
        """
        from io import BytesIO

        pil = Image.open(BytesIO(image_bytes)).convert("RGB")
        return self.match_pil(pil, museum=museum, topk=topk)


main.py

In [ ]:
"""
EAR Backend API (FastAPI + Uvicorn)

Exposes:
- GET  /health     → quick status check
- POST /match      → upload an image and get Top-K matched artworks

It uses the PoseMatcher defined in backend/model/pose_matcher.py and the
Settings from backend/config.py.

Default index: data/mixed/ (global mixed index: local + met).
"""

from __future__ import annotations

import io
import logging
from typing import List, Optional

from fastapi import FastAPI, File, Form, HTTPException, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from PIL import Image

from .config import Settings
from .model.pose_matcher import PoseMatcher


logger = logging.getLogger("uvicorn.error")

# ----------------------------------------------------------------------
# FastAPI app + CORS
# ----------------------------------------------------------------------

app = FastAPI(
    title="Embodied Aesthetic Reconstruction — Backend API",
    version="0.2.0",
    description="CLIP + pose based portrait → artwork matching backend.",
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # you can tighten this later
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global objects filled at startup
settings: Optional[Settings] = None
matcher: Optional[PoseMatcher] = None


# ----------------------------------------------------------------------
# Startup / shutdown
# ----------------------------------------------------------------------


@app.on_event("startup")
def _startup_load() -> None:
    """Load configuration + matcher once when the server starts."""
    global settings, matcher

    logger.info("Loading Settings and PoseMatcher…")
    settings = Settings()
    matcher = PoseMatcher(settings)
    logger.info("Matcher loaded successfully.")


@app.on_event("shutdown")
def _shutdown_cleanup() -> None:
    """Optional cleanup (currently nothing special)."""
    global matcher
    matcher = None
    logger.info("Matcher released.")


# ----------------------------------------------------------------------
# Simple health endpoint
# ----------------------------------------------------------------------


@app.get("/health")
def health():
    return {
        "status": "ok",
        "device": getattr(matcher, "device", "unknown") if matcher else "unloaded",
        "default_museum": getattr(settings, "default_museum", None)
        if settings
        else None,
        "pose_weight": getattr(settings, "pose_weight", None) if settings else None,
    }


# ----------------------------------------------------------------------
# Schemas for /match
# ----------------------------------------------------------------------


class MatchResult(BaseModel):
    filename: str
    title: Optional[str] = None
    artist: Optional[str] = None
    year: Optional[int] = None
    score: float


class MatchResponse(BaseModel):
    museum: Optional[str]
    topk: int
    results: List[MatchResult]


# ----------------------------------------------------------------------
# Main matching endpoint
# ----------------------------------------------------------------------


@app.post("/match", response_model=MatchResponse)
async def match(
    image: UploadFile = File(..., description="Uploaded portrait photo"),
    museum: Optional[str] = Form(None, description="Museum name (kept for compatibility)"),
    topk: int = Form(3, description="Number of top results"),
):
    """
    Robust matching endpoint:

    - validates that the uploaded file looks like an image
    - decodes it to a PIL image (RGB)
    - calls PoseMatcher.match_pil(...)
    - tries to back-fill missing title / artist / year from matcher.meta_by_filename
    """
    if topk <= 0:
        topk = 3

    if image.content_type is not None and not image.content_type.startswith("image/"):
        raise HTTPException(
            status_code=400,
            detail=f"Unsupported content type: {image.content_type}. Please upload an image file.",
        )

    data = await image.read()
    if not data:
        raise HTTPException(
            status_code=400,
            detail="Empty file received. Please take a photo again.",
        )

    try:
        pil_img = Image.open(io.BytesIO(data)).convert("RGB")
    except Exception as exc:
        logger.exception("Failed to decode uploaded image")
        raise HTTPException(
            status_code=400,
            detail=f"Invalid image file: {type(exc).__name__}",
        )

    global matcher
    if matcher is None:
        raise HTTPException(
            status_code=500,
            detail="Matcher is not loaded on the server.",
        )

    effective_museum = museum or getattr(matcher, "default_museum", None) or "mixed"

    try:
        if hasattr(matcher, "match_pil"):
            raw_results = matcher.match_pil(
                pil_img,
                museum=effective_museum,
                topk=topk,
            )
        else:
            raise RuntimeError("PoseMatcher has no method 'match_pil'")
    except ValueError as exc:
        logger.exception("ValueError in matcher.match_pil")
        raise HTTPException(
            status_code=400,
            detail=f"Matching failed: {exc}",
        )
    except Exception as exc:
        logger.exception("Unhandled error in matcher.match_pil")
        raise HTTPException(
            status_code=500,
            detail=f"Internal error during matching: {type(exc).__name__}: {exc}",
        )

    payload: List[MatchResult] = []

    meta_store = getattr(matcher, "meta_by_filename", None)

    def fill_from_meta(filename: str, title, artist, year):
        """If title/artist/year are missing, try to fill from matcher metadata."""
        nonlocal meta_store
        if not isinstance(meta_store, dict):
            return title, artist, year

        meta = meta_store.get(filename) or meta_store.get(filename.strip()) or {}
        if isinstance(meta, dict):
            title = title or meta.get("title") or meta.get("Title")
            artist = artist or meta.get("artist") or meta.get("Artist")
            year = year or meta.get("year") or meta.get("Year")
        else:
            title = title or getattr(meta, "title", None)
            artist = artist or getattr(meta, "artist", None)
            year = year or getattr(meta, "year", None)
        return title, artist, year

    for r in raw_results or []:
        if isinstance(r, dict):
            filename = r.get("filename") or r.get("file") or ""
            score = float(r.get("score", 0.0))
            title = r.get("title")
            artist = r.get("artist")
            year = r.get("year")
        else:
            filename = getattr(r, "filename", "") or getattr(r, "file", "")
            score = float(getattr(r, "score", 0.0))
            title = getattr(r, "title", None)
            artist = getattr(r, "artist", None)
            year = getattr(r, "year", None)

        title, artist, year = fill_from_meta(filename, title, artist, year)

        # normalise year → int or None
        try:
            if isinstance(year, str) and year.strip():
                year_int = int(year.split(",")[0].strip())
            elif isinstance(year, (int, float)):
                year_int = int(year)
            else:
                year_int = None
        except Exception:
            year_int = None

        payload.append(
            MatchResult(
                filename=filename,
                title=title,
                artist=artist,
                year=year_int,
                score=score,
            )
        )

    return MatchResponse(museum=effective_museum, topk=topk, results=payload)


pose_matcher.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Hybrid Pose+CLIP Matcher
------------------------
This module performs a hybrid similarity search over a museum dataset.

Data folder layout (per museum):
    data/<museum>/
        embeddings.npy          # (N, D_clip)
        pose_embeddings.npy     # (N, D_pose)   ← optional
        embeddings_meta.csv     # filenames + metadata

Final score:
    final = (1 - POSE_WEIGHT) * clip_sim + (POSE_WEIGHT) * pose_sim
"""

import os
import json
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional, Tuple

from .utils import normalize_rows, cosine_similarity_matrix


class PoseMatcher:
    """
    Hybrid matcher combining CLIP embeddings + Pose embeddings.
    """

    def __init__(
        self,
        museum_dir: Path,
        pose_weight: float = 0.35,        # You can tune this
        topk_default: int = 3,
    ):
        self.museum_dir = museum_dir
        self.pose_weight = float(pose_weight)
        self.topk_default = int(topk_default)

        # Load embeddings
        emb_path = museum_dir / "embeddings.npy"
        meta_path = museum_dir / "embeddings_meta.csv"
        pose_path = museum_dir / "pose_embeddings.npy"

        if not emb_path.exists():
            raise FileNotFoundError(f"Missing embeddings: {emb_path}")

        self.clip_emb = np.load(emb_path).astype("float32")
        self.clip_emb = normalize_rows(self.clip_emb)

        # Metadata
        self.meta = self._load_meta_rows(meta_path)

        # Pose embeddings (optional)
        if pose_path.exists():
            print(f"[INFO] Pose embeddings found at {pose_path}")
            self.pose_emb = np.load(pose_path).astype("float32")
            self.pose_emb = normalize_rows(self.pose_emb)
            self.has_pose = True
        else:
            print(f"[INFO] No pose_embeddings.npy → using CLIP-only mode")
            self.pose_emb = None
            self.has_pose = False

        # Basic sanity checks
        if len(self.meta) != len(self.clip_emb):
            raise ValueError(
                f"Metadata length {len(self.meta)} != embeddings length {len(self.clip_emb)}"
            )
        if self.has_pose and len(self.pose_emb) != len(self.clip_emb):
            raise ValueError(
                f"Pose embeddings length {len(self.pose_emb)} != CLIP embeddings length {len(self.clip_emb)}"
            )

        print(
            f"[OK] Loaded museum='{museum_dir.name}' "
            f"→ N={len(self.clip_emb)}, pose={self.has_pose}"
        )

    # -------------------------------------------------------------
    def _load_meta_rows(self, path: Path) -> List[Dict]:
        import csv

        if not path.exists():
            raise FileNotFoundError(f"Metadata CSV not found: {path}")

        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for row in csv.DictReader(f):
                rows.append(row)
        return rows

    # -------------------------------------------------------------
    def encode_query_image(self, pil, clip_model, preprocess, device):
        """
        Encode query image using CLIP image encoder.
        """
        import torch

        x = preprocess(pil).unsqueeze(0).to(device)
        with torch.no_grad(), torch.autocast(device_type=str(device), enabled=(device.type in ["cuda", "mps"])):
            feat = clip_model.encode_image(x)
        feat = feat.float().cpu().numpy()
        feat = normalize_rows(feat)
        return feat  # (1, D)

    # -------------------------------------------------------------
    def compute_pose_for_query(self, kp_vec: Optional[np.ndarray]) -> Optional[np.ndarray]:
        """
        You pass in a (D_pose,) vector computed from YOLO keypoints.
        This function simply normalizes it.
        """
        if kp_vec is None or not self.has_pose:
            return None
        v = kp_vec.astype("float32")[None, :]
        v = normalize_rows(v)
        return v  # (1, D_pose)

    # -------------------------------------------------------------
    def match(
        self,
        clip_query_vec: np.ndarray,
        pose_query_vec: Optional[np.ndarray] = None,
        topk: Optional[int] = None,
    ) -> List[Dict]:

        if topk is None:
            topk = self.topk_default

        # --- 1) CLIP similarity
        sim_clip = (self.clip_emb @ clip_query_vec.T).reshape(-1)

        # --- 2) Pose similarity (if enabled)
        if self.has_pose and pose_query_vec is not None:
            sim_pose = (self.pose_emb @ pose_query_vec.T).reshape(-1)
        else:
            sim_pose = None

        # --- 3) Hybrid scoring
        if sim_pose is not None:
            w = self.pose_weight
            final = (1.0 - w) * sim_clip + w * sim_pose
        else:
            final = sim_clip

        # --- 4) Sort
        idx = np.argsort(final)[::-1][:topk]

        # --- 5) Pack results
        out = []
        for i in idx:
            row = self.meta[i]
            out.append(
                dict(
                    filename=row.get("filename") or row.get("file"),
                    score=float(final[i]),
                    clip=float(sim_clip[i]),
                    pose=float(sim_pose[i]) if sim_pose is not None else None,
                    meta=row,
                )
            )
        return out


app_frontend.py

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
import random
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# Meta CSV candidates (fallback to local if needed)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

DEFAULT_API_URL = "http://127.0.0.1:8000/match"
APP_TITLE = "Embodied Aesthetic Reconstruction"

YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900

# Diversity / randomness control
MAX_RECENT = 6          # how many artworks to remember
PRIMARY_TOP_K = 3       # "very best" range
TAIL_TOP_K = 10         # search in top-10 in total
TAIL_RANDOM_PROB = 0.3  # 30% chance to look into 4–10 first


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    if not filename:
        return None
    p = Path(filename)
    if not p.is_absolute():
        p = IMAGES_DIR / p
    return p if p.exists() else None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """Small pink pose metrics in top-right corner of the image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """Yellow label stack: price, year, artist."""
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}

body {{
  background-color: #f5f5f7;
}}
.block-container {{
  padding-top: 0.6rem;
  padding-bottom: 0.6rem;
  max-width: 1700px;
}}

h1 {{
  font-family: -apple-system, BlinkMacSystemFont, "SF Pro Display", system-ui, sans-serif;
  letter-spacing: 0.04em;
  font-weight: 700;
}}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 18px;
  background: #111;
  box-shadow: 0 18px 45px rgba(0,0,0,0.3);
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 18px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
  border-radius: 18px;
  background: #050505;
  box-shadow: 0 18px 45px rgba(0,0,0,0.35);
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}

button[kind="secondary"] {{
  border-radius: 999px !important;
}}

</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink pose metrics."
)

try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0
if "recent_files" not in st.session_state:
    st.session_state["recent_files"] = []  # type: List[str]

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")
    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # Ask backend for Top-10, we'll do diversity selection here
            data = {"museum": "mixed", "topk": TAIL_TOP_K}

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                recent = st.session_state.get("recent_files", [])

                # ---------- diversity + mild randomness ----------
                # Decide whether to start looking from the tail (ranks 4–10)
                use_tail_first = random.random() < TAIL_RANDOM_PROB and len(results) > PRIMARY_TOP_K
                primary_slice = results[:PRIMARY_TOP_K]
                tail_slice = results[PRIMARY_TOP_K:TAIL_TOP_K]

                ordered_candidates = (
                    (tail_slice + primary_slice) if use_tail_first else (primary_slice + tail_slice)
                )

                chosen = None
                chosen_fname = None

                for r in ordered_candidates:
                    fname = (
                        r.get("filename")
                        or r.get("file")
                        or r.get("image_path")
                        or r.get("path")
                    )
                    if not fname:
                        continue
                    if fname not in recent:
                        chosen = r
                        chosen_fname = fname
                        break

                # Fallback: if all candidates are "recent", use strict Top-1
                if chosen is None:
                    chosen = results[0]
                    chosen_fname = (
                        chosen.get("filename")
                        or chosen.get("file")
                        or chosen.get("image_path")
                        or chosen.get("path")
                    )

                # Update recent memory
                if chosen_fname:
                    if chosen_fname in recent:
                        recent.remove(chosen_fname)
                    recent.insert(0, chosen_fname)
                    del recent[MAX_RECENT:]
                    st.session_state["recent_files"] = recent

                filename = chosen_fname

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or chosen.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or chosen.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(painted, metrics, size=16, margin=12)

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


app_frontend.py

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
import random
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met + aic merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# All museum image roots (search order)
MUSEUM_IMAGE_DIRS = [
    DATA_DIR / "images",                    # mixed
    ROOT_DIR / "data" / "local" / "images",
    ROOT_DIR / "data" / "met" / "images",
    ROOT_DIR / "data" / "aic" / "images",
]

# Meta CSV candidates (fallback to local if needed)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

DEFAULT_API_URL = "http://127.0.0.1:8000/match"
APP_TITLE = "Embodied Aesthetic Reconstruction"

YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900

# Diversity / randomness control
MAX_RECENT = 6          # how many artworks to remember
PRIMARY_TOP_K = 3       # "very best" range
TAIL_TOP_K = 10         # search in top-10 in total
TAIL_RANDOM_PROB = 0.3  # 30% chance to look into 4–10 first


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """
    Resolve an artwork filename to a real image path.

    Strategy:
      1) If it's an absolute existing path → return.
      2) If it has subdirectories, try relative to project root / data.
      3) Otherwise, search by basename in all known museum image dirs.
    """
    if not filename:
        return None

    p = Path(filename)

    # 1) absolute path already
    if p.is_absolute() and p.exists():
        return p

    candidates: List[Path] = []

    # 2) has subdirectory component (e.g. "aic/images/aic_4939.jpg")
    if p.parent != Path("."):
        candidates.append(ROOT_DIR / p)
        candidates.append(ROOT_DIR / "data" / p)

    # 3) search by basename across all museum image roots
    basename = p.name
    for root in MUSEUM_IMAGE_DIRS:
        candidates.append(root / basename)

    for c in candidates:
        if c.exists():
            return c

    return None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """Small pink pose metrics in top-right corner of the image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """Yellow label stack: price, year, artist."""
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}

body {{
  background-color: #f5f5f7;
}}
.block-container {{
  padding-top: 0.6rem;
  padding-bottom: 0.6rem;
  max-width: 1700px;
}}

h1 {{
  font-family: -apple-system, BlinkMacSystemFont, "SF Pro Display", system-ui, sans-serif;
  letter-spacing: 0.04em;
  font-weight: 700;
}}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 18px;
  background: #111;
  box-shadow: 0 18px 45px rgba(0,0,0,0.3);
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 18px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
  border-radius: 18px;
  background: #050505;
  box-shadow: 0 18px 45px rgba(0,0,0,0.35);
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}

button[kind="secondary"] {{
  border-radius: 999px !important;
}}

</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink pose metrics."
)

try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0
if "recent_files" not in st.session_state:
    st.session_state["recent_files"] = []  # type: List[str]

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")

    # Match mode control
    match_mode = st.selectbox(
        "Match mode",
        ["default", "portrait_only", "high_value_only", "portrait_high_value"],
        index=0,
        help="Use 'portrait_only' to bias towards clear portraits; "
             "'high_value_only' for masterpieces.",
    )

    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # Ask backend for Top-10, we'll do diversity selection here
            data = {
                "museum": "mixed",
                "topk": TAIL_TOP_K,
                "mode": match_mode,
            }

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                recent = st.session_state.get("recent_files", [])

                # ---------- diversity + mild randomness ----------
                # Decide whether to start looking from the tail (ranks 4–10)
                use_tail_first = (
                    random.random() < TAIL_RANDOM_PROB
                    and len(results) > PRIMARY_TOP_K
                )
                primary_slice = results[:PRIMARY_TOP_K]
                tail_slice = results[PRIMARY_TOP_K:TAIL_TOP_K]

                ordered_candidates = (
                    (tail_slice + primary_slice)
                    if use_tail_first
                    else (primary_slice + tail_slice)
                )

                chosen = None
                chosen_fname = None

                for r in ordered_candidates:
                    fname = (
                        r.get("filename")
                        or r.get("file")
                        or r.get("image_path")
                        or r.get("path")
                    )
                    if not fname:
                        continue
                    if fname not in recent:
                        chosen = r
                        chosen_fname = fname
                        break

                # Fallback: if all candidates are "recent", use strict Top-1
                if chosen is None:
                    chosen = results[0]
                    chosen_fname = (
                        chosen.get("filename")
                        or chosen.get("file")
                        or chosen.get("image_path")
                        or chosen.get("path")
                    )

                # Update recent memory
                if chosen_fname:
                    if chosen_fname in recent:
                        recent.remove(chosen_fname)
                    recent.insert(0, chosen_fname)
                    del recent[MAX_RECENT:]
                    st.session_state["recent_files"] = recent

                filename = chosen_fname

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or chosen.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or chosen.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


tools/build_pose_embeddings.py(clip_version)

In [ ]:
"""
build_pose_embeddings.py
-------------------------
Generate pose-based embeddings for all artworks using YOLOv8-Pose.

Output:
    data/mixed/pose_embeddings.npy
    (same row ordering as embeddings_meta.csv)

Usage:
    conda activate ear-mvp
    python backend/tools/build_pose_embeddings.py --data-root data/mixed
"""

from __future__ import annotations

import argparse
from pathlib import Path
import numpy as np
from PIL import Image
import json
from tqdm import tqdm

# YOLOv8-Pose
try:
    from ultralytics import YOLO
except Exception as exc:
    raise RuntimeError("Ultralytics YOLO is required. `pip install ultralytics`.") from exc


# -------------------------
# Pose vector utilities
# -------------------------

def encode_pose_keypoints(kps: np.ndarray | None, img_w: int, img_h: int) -> np.ndarray:
    """
    Convert raw keypoints (N x 3: x,y,confidence) into a fixed pose vector.

    We encode:
      - normalized coordinates (x/img_w, y/img_h)
      - per-joint existence mask
      - joint angle features
    """

    num_joints = 17  # YOLOv8 Pose has 17 keypoints (0..16)
    if kps is None:
        # No detection → zero vector
        return np.zeros(num_joints * 4, dtype=np.float32)

    # If YOLO returns more than one person, we take the one with largest area
    # but build_embeddings will only run on artwork images (static), so 1 person is expected.
    if len(kps.shape) == 3:  # shape [num_people, 17, 3]
        kps = kps[0]

    # Prepare vector components
    coord_vec = np.zeros((num_joints, 2), dtype=np.float32)
    mask_vec  = np.zeros((num_joints, 1), dtype=np.float32)
    angle_vec = np.zeros((num_joints, 1), dtype=np.float32)

    # Normalize
    for i in range(num_joints):
        x, y, conf = kps[i]
        if conf > 0.1:
            coord_vec[i] = np.array([x / img_w, y / img_h])
            mask_vec[i] = 1.0
        else:
            coord_vec[i] = 0.0
            mask_vec[i] = 0.0

    # Compute simple limb angles (example: shoulders/elbows/wrists)
    # We encode angle at elbow and knee joints for stability
    def safe_angle(a, b, c):
        """Return angle ABC in radians."""
        if (a is None) or (b is None) or (c is None):
            return 0.0
        ax, ay = a
        bx, by = b
        cx, cy = c
        v1 = np.array([ax - bx, ay - by])
        v2 = np.array([cx - bx, cy - by])
        n1 = np.linalg.norm(v1)
        n2 = np.linalg.norm(v2)
        if n1 < 1e-6 or n2 < 1e-6:
            return 0.0
        cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
        return float(np.arccos(cosv))  # radians

    # Joint indices follow COCO:
    # 5: left shoulder, 6: right shoulder
    # 7: left elbow,    8: right elbow
    # 9: left wrist,   10: right wrist
    # 11: left hip,    12: right hip
    # 13: left knee,   14: right knee
    # 15: left ankle,  16: right ankle

    joints = kps[:, :2]
    def jp(idx):
        x, y, conf = kps[idx]
        return (x, y) if conf > 0.1 else None

    # elbow angles
    angle_vec[7]  = safe_angle(jp(5), jp(7), jp(9))
    angle_vec[8]  = safe_angle(jp(6), jp(8), jp(10))

    # knee angles
    angle_vec[13] = safe_angle(jp(11), jp(13), jp(15))
    angle_vec[14] = safe_angle(jp(12), jp(14), jp(16))

    # Flatten
    return np.concatenate([
        coord_vec.reshape(-1),
        mask_vec.reshape(-1),
        angle_vec.reshape(-1),
    ]).astype(np.float32)


# -------------------------
# Main build function
# -------------------------

def build_pose_embeddings(data_root: Path, model_path: Path):
    meta_path = data_root / "embeddings_meta.csv"
    images_dir = data_root / "images"

    if not meta_path.exists():
        raise FileNotFoundError(f"Missing: {meta_path}")

    # Load metadata rows in order
    import csv
    with open(meta_path, "r", encoding="utf-8") as f:
        rows = list(csv.DictReader(f))

    print(f"[INFO] Loaded {len(rows)} metadata rows.")

    # Init YOLO pose model
    print(f"[INFO] Loading YOLOv8-Pose: {model_path}")
    model = YOLO(str(model_path))

    pose_vectors = []

    for row in tqdm(rows, desc="Building pose vectors"):
        fname = (
            row.get("filename")
            or row.get("image_path")
            or row.get("path")
            or row.get("file")
        )
        if not fname:
            pose_vectors.append(np.zeros(17 * 4, dtype=np.float32))
            continue

        img_path = images_dir / fname
        if not img_path.exists():
            pose_vectors.append(np.zeros(17 * 4, dtype=np.float32))
            continue

        try:
            img = Image.open(img_path).convert("RGB")
        except Exception:
            pose_vectors.append(np.zeros(17 * 4, dtype=np.float32))
            continue

        w, h = img.size
        img_np = np.array(img)

        # YOLO pose inference
        res = model.predict(img_np, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].keypoints is None:
            pose_vectors.append(np.zeros(17 * 4, dtype=np.float32))
            continue

        kps = res[0].keypoints.cpu().numpy()  # shape: [1, 17, 3]
        vec = encode_pose_keypoints(kps, w, h)
        pose_vectors.append(vec)

    pose_vectors = np.stack(pose_vectors, axis=0)
    out_path = data_root / "pose_embeddings.npy"

    np.save(out_path, pose_vectors)
    print(f"[OK] Saved pose embeddings → {out_path}")
    print(f"[INFO] shape = {pose_vectors.shape} (rows match embeddings_meta.csv)")


# -------------------------
# CLI
# -------------------------

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--data-root", type=str, default="data/mixed",
                        help="path to dataset folder that contains images/ and embeddings_meta.csv")
    parser.add_argument("--yolo-model", type=str, default="frontend/yolov8n-pose.pt",
                        help="path to YOLOv8-Pose .pt file")

    args = parser.parse_args()

    build_pose_embeddings(
        Path(args.data_root),
        Path(args.yolo_model),
    )


backend/utils_pose.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
utils_pose.py
-------------
Unified utilities to convert YOLOv8-Pose keypoints → pose embedding vectors.

This MUST match build_pose_embeddings.py so that query vectors
are compatible with stored pose_embeddings.npy.
"""

import numpy as np


def encode_keypoints_to_pose_vector(
    keypoints_xy: np.ndarray,   # shape: (17, 2)
    visibility: np.ndarray,     # shape: (17,)
) -> np.ndarray:
    """
    Convert YOLOv8 keypoints → a normalized pose embedding vector.

    Steps:
      1. Normalize XY to [0,1] by image width/height (caller must pre-normalize)
      2. Use visibility as mask
      3. Compute limb angles
      4. Flatten into 1D feature vector
    """

    # ----- 1) Normalize XY (expected by build_pose_embeddings)
    xy = keypoints_xy.astype("float32")
    vis = visibility.astype("float32")

    # Flatten XY
    xy_flat = xy.reshape(-1)  # (34,)

    # ----- 2) visibility mask
    mask = vis.reshape(-1)  # (17,)

    # ----- 3) Compute simple limb angles
    def angle(p1, p2):
        if p1 is None or p2 is None:
            return 0.0
        vx, vy = p2[0] - p1[0], p2[1] - p1[1]
        return np.degrees(np.arctan2(vy, vx))

    # Example pairs (shoulder-left → elbow-left, etc.)
    k = xy
    angle_pairs = [
        (5, 7),
        (7, 9),
        (6, 8),
        (8, 10),
    ]
    angs = []
    for a, b in angle_pairs:
        p1 = k[a] if vis[a] > 0.1 else None
        p2 = k[b] if vis[b] > 0.1 else None
        angs.append(angle(p1, p2))
    angs = np.array(angs, dtype="float32")  # (4,)

    # ----- 4) Concatenate all
    feat = np.concatenate([
        xy_flat,      # 34
        mask,         # 17
        angs,         # 4
    ]).astype("float32")  # total = 34+17+4 = 55 dims

    return feat  # (55,)


app_frontend.py(736)

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
import random
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met + aic merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# All museum image roots (search order)
MUSEUM_IMAGE_DIRS = [
    DATA_DIR / "images",                    # mixed
    ROOT_DIR / "data" / "local" / "images",
    ROOT_DIR / "data" / "met" / "images",
    ROOT_DIR / "data" / "aic" / "images",
]

# Meta CSV candidates (fallback to local if needed)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

DEFAULT_API_URL = "http://127.0.0.1:8000/match"
APP_TITLE = "Embodied Aesthetic Reconstruction"

YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900

# Diversity / randomness control
MAX_RECENT = 6          # how many artworks to remember
PRIMARY_TOP_K = 3       # "very best" range
TAIL_TOP_K = 10         # search in top-10 in total
TAIL_RANDOM_PROB = 0.3  # 30% chance to look into 4–10 first


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """
    Resolve an artwork filename to a real image path.

    Strategy:
      1) If it's an absolute existing path → return.
      2) If it has subdirectories, try relative to project root / data.
      3) Otherwise, search by basename in all known museum image dirs.
    """
    if not filename:
        return None

    p = Path(filename)

    # 1) absolute path already
    if p.is_absolute() and p.exists():
        return p

    candidates: List[Path] = []

    # 2) has subdirectory component (e.g. "aic/images/aic_4939.jpg")
    if p.parent != Path("."):
        candidates.append(ROOT_DIR / p)
        candidates.append(ROOT_DIR / "data" / p)

    # 3) search by basename across all museum image roots
    basename = p.name
    for root in MUSEUM_IMAGE_DIRS:
        candidates.append(root / basename)

    for c in candidates:
        if c.exists():
            return c

    return None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """Small pink pose metrics in top-right corner of the image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """Yellow label stack: price, year, artist."""
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}

body {{
  background-color: #f5f5f7;
}}
.block-container {{
  padding-top: 0.6rem;
  padding-bottom: 0.6rem;
  max-width: 1700px;
}}

h1 {{
  font-family: -apple-system, BlinkMacSystemFont, "SF Pro Display", system-ui, sans-serif;
  letter-spacing: 0.04em;
  font-weight: 700;
}}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 18px;
  background: #111;
  box-shadow: 0 18px 45px rgba(0,0,0,0.3);
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 18px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
  border-radius: 18px;
  background: #050505;
  box-shadow: 0 18px 45px rgba(0,0,0,0.35);
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}

button[kind="secondary"] {{
  border-radius: 999px !important;
}}

</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink pose metrics."
)

try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0
if "recent_files" not in st.session_state:
    st.session_state["recent_files"] = []  # type: List[str]

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")

    # Match mode control
    match_mode = st.selectbox(
        "Match mode",
        ["default", "portrait_only", "high_value_only", "portrait_high_value"],
        index=0,
        help="Use 'portrait_only' to bias towards clear portraits; "
             "'high_value_only' for masterpieces.",
    )

    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # Ask backend for Top-10, we'll do diversity selection here
            data = {
                "museum": "mixed",
                "topk": TAIL_TOP_K,
                "mode": match_mode,
            }

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                recent = st.session_state.get("recent_files", [])

                # ---------- diversity + mild randomness ----------
                # Decide whether to start looking from the tail (ranks 4–10)
                use_tail_first = (
                    random.random() < TAIL_RANDOM_PROB
                    and len(results) > PRIMARY_TOP_K
                )
                primary_slice = results[:PRIMARY_TOP_K]
                tail_slice = results[PRIMARY_TOP_K:TAIL_TOP_K]

                ordered_candidates = (
                    (tail_slice + primary_slice)
                    if use_tail_first
                    else (primary_slice + tail_slice)
                )

                chosen = None
                chosen_fname = None

                for r in ordered_candidates:
                    fname = (
                        r.get("filename")
                        or r.get("file")
                        or r.get("image_path")
                        or r.get("path")
                    )
                    if not fname:
                        continue
                    if fname not in recent:
                        chosen = r
                        chosen_fname = fname
                        break

                # Fallback: if all candidates are "recent", use strict Top-1
                if chosen is None:
                    chosen = results[0]
                    chosen_fname = (
                        chosen.get("filename")
                        or chosen.get("file")
                        or chosen.get("image_path")
                        or chosen.get("path")
                    )

                # Update recent memory
                if chosen_fname:
                    if chosen_fname in recent:
                        recent.remove(chosen_fname)
                    recent.insert(0, chosen_fname)
                    del recent[MAX_RECENT:]
                    st.session_state["recent_files"] = recent

                filename = chosen_fname

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or chosen.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or chosen.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


app_frontend.py(737)

In [ ]:
from __future__ import annotations

import io
import os
import time
import threading
import random
from collections import deque
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import streamlit as st
from streamlit_webrtc import (
    WebRtcMode,
    RTCConfiguration,
    VideoProcessorBase,
    webrtc_streamer,
)

# =================== PATHS & CONSTANTS ===================

FRONTEND_DIR = Path(__file__).resolve().parent
ROOT_DIR = FRONTEND_DIR.parent

# Use GLOBAL MIXED INDEX (local + met + aic merged)
DATA_DIR = ROOT_DIR / "data" / "mixed"
IMAGES_DIR = DATA_DIR / "images"

# All museum image roots (search order)
MUSEUM_IMAGE_DIRS = [
    DATA_DIR / "images",                    # mixed
    ROOT_DIR / "data" / "local" / "images",
    ROOT_DIR / "data" / "met" / "images",
    ROOT_DIR / "data" / "aic" / "images",
]

# Meta CSV candidates (fallback to local if needed)
META_CSV_CANDIDATES = [
    DATA_DIR / "embeddings_meta.csv",
    ROOT_DIR / "data" / "local" / "portrait_works_enhanced_english.csv",
    ROOT_DIR / "data" / "local" / "portrait_works.csv",
]

DEFAULT_API_URL = "http://127.0.0.1:8000/match"
APP_TITLE = "Embodied Aesthetic Reconstruction"

YOLO_MODEL_PATH = FRONTEND_DIR / "yolov8n-pose.pt"

RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# Stillness / motion parameters
STILLNESS_SEC = 3.5
MAX_BUF_SEC = 5.0
FPS_ASSUMED = 12
MOTION_EPS_CXCY = 4.0
MOTION_EPS_AREA = 0.03
MIN_FACE_AREA = 0.06

# Colors
YELLOW = (255, 235, 59)
BLACK = (0, 0, 0)
HOT_PINK = (255, 30, 180)

RIGHT_IMG_MAXW = 900

# Diversity / randomness control
MAX_RECENT = 6          # how many artworks to remember
PRIMARY_TOP_K = 3       # "very best" range
TAIL_TOP_K = 10         # search in top-10 in total
TAIL_RANDOM_PROB = 0.3  # 30% chance to look into 4–10 first


# =================== UTILITIES ===================

_META_CACHE: Optional[Dict[str, Dict]] = None


def load_meta_mapping() -> Dict[str, Dict]:
    """Load CSV → filename → metadata mapping."""
    global _META_CACHE
    if _META_CACHE is not None:
        return _META_CACHE

    import csv

    rows: List[Dict] = []
    for p in META_CSV_CANDIDATES:
        if p.exists():
            with open(p, "r", encoding="utf-8") as f:
                rows = list(csv.DictReader(f))
            break

    mapping: Dict[str, Dict] = {}
    for r in rows:
        fname = (
            r.get("filename")
            or r.get("image_path")
            or r.get("path")
            or r.get("file")
        )
        if fname:
            mapping[str(fname)] = r

    _META_CACHE = mapping
    return mapping


def lookup_meta(filename: str) -> Dict:
    mapping = load_meta_mapping()
    return mapping.get(filename, {})


def safe_open_image(p: Path) -> Optional[Image.Image]:
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        return None


def ensure_image_path(filename: str) -> Optional[Path]:
    """
    Resolve an artwork filename to a real image path.

    Strategy:
      1) If it's an absolute existing path → return.
      2) If it has subdirectories, try relative to project root / data.
      3) Otherwise, search by basename in all known museum image dirs.
    """
    if not filename:
        return None

    p = Path(filename)

    # 1) absolute path already
    if p.is_absolute() and p.exists():
        return p

    candidates: List[Path] = []

    # 2) has subdirectory component (e.g. "aic/images/aic_4939.jpg")
    if p.parent != Path("."):
        candidates.append(ROOT_DIR / p)
        candidates.append(ROOT_DIR / "data" / p)

    # 3) search by basename across all museum image roots
    basename = p.name
    for root in MUSEUM_IMAGE_DIRS:
        candidates.append(root / basename)

    for c in candidates:
        if c.exists():
            return c

    return None


def _load_font(size: int = 40):
    """Try Courier → Arial → default."""
    candidates = [
        "/Library/Fonts/Courier New.ttf",
        "/System/Library/Fonts/Courier.dfont",
        "/System/Library/Fonts/Supplemental/Courier New.ttf",
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                return ImageFont.truetype(p, size=size)
            except Exception:
                continue
    return ImageFont.load_default()


def draw_tiny_metrics_top_right(
    im: Image.Image, lines: List[str], size: int = 16, margin: int = 10
) -> Image.Image:
    """Small pink pose metrics in top-right corner of the image."""
    if not lines:
        return im
    img = im.copy()
    d = ImageDraw.Draw(img)
    font = _load_font(size)

    widths = []
    for s in lines:
        l, t, r, b = d.textbbox((0, 0), s, font=font)
        widths.append(r - l)
    wmax = max(widths) if widths else 0

    x = img.width - margin - wmax
    y = margin
    for s in lines:
        d.text((x, y), s, fill=HOT_PINK, font=font)
        _, _, _, b = d.textbbox((0, 0), s, font=font)
        y += int(b * 0.95)
    return img


def _angle_deg(p1, p2):
    if p1 is None or p2 is None:
        return None
    vx, vy = p2[0] - p1[0], p2[1] - p1[1]
    return float(np.degrees(np.arctan2(vy, vx)))


def _elbow_angle(shoulder, elbow, wrist):
    if None in (shoulder, elbow, wrist):
        return None
    v1 = np.array([shoulder[0] - elbow[0], shoulder[1] - elbow[1]], float)
    v2 = np.array([wrist[0] - elbow[0], wrist[1] - elbow[1]], float)
    n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if n1 < 1e-5 or n2 < 1e-5:
        return None
    cosv = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))


def format_metrics(kps: Dict[int, Tuple[float, float] | None]) -> List[str]:
    le, re = kps.get(1), kps.get(2)
    lsh, rsh = kps.get(5), kps.get(6)
    lel, rel = kps.get(7), kps.get(8)
    lwr, rwr = kps.get(9), kps.get(10)

    fdeg = lambda v: "—" if v is None else f"{v:+.1f}°"
    fpt = lambda p: "(0, 0)" if p is None else f"({int(p[0])}, {int(p[1])})"

    return [
        f"Head tilt: {fdeg(_angle_deg(re, le))}",
        f"Shoulder:  {fdeg(_angle_deg(rsh, lsh))}",
        f"L elbow:   {fdeg(_elbow_angle(lsh, lel, lwr))}",
        f"R elbow:   {fdeg(_elbow_angle(rsh, rel, rwr))}",
        f"L wrist:   {fpt(lwr)}",
        f"R wrist:   {fpt(rwr)}",
    ]


def overlay_right_labels(painting: Image.Image, meta: Dict) -> Image.Image:
    """Yellow label stack: price, year, artist."""
    im = painting.convert("RGB").copy()
    draw = ImageDraw.Draw(im)

    font_big = _load_font(44)
    font_small = _load_font(36)

    price = (
        meta.get("price_text")
        or meta.get("auction_price_usd")
        or meta.get("price")
        or "—"
    )
    year = str(meta.get("year") or "—")
    artist = meta.get("artist") or "artist name"

    lines = [price, year, artist]
    fonts = [font_big, font_small, font_big]

    margin_x = 24
    y = int(im.height * 0.50)

    for text, font in zip(lines, fonts):
        l, t, r, b = draw.textbbox((0, 0), text, font=font)
        w, h = r - l, b - t

        pad_x, pad_y = 16, 10
        box_w, box_h = w + 2 * pad_x, h + 2 * pad_y

        x = margin_x
        draw.rectangle([x, y, x + box_w, y + box_h], fill=YELLOW)
        draw.text((x + pad_x, y + pad_y), text, fill=BLACK, font=font)

        y += box_h + 10

    return im


def force_rerun():
    """Compatible with new/old Streamlit."""
    try:
        st.rerun()
    except Exception:
        try:
            st.experimental_rerun()
        except Exception:
            pass


# =================== YOLO VIDEO PROCESSOR ===================

try:
    from ultralytics import YOLO

    HAS_YOLO = True
except Exception:
    HAS_YOLO = False


class CuratorialProcessor(VideoProcessorBase):
    """
    YOLOv8-Pose:
    - blue bbox
    - green skeleton
    - pink tiny pose metrics
    - stillness-based auto capture
    """

    def __init__(self):
        self.model = None
        if HAS_YOLO:
            try:
                local = YOLO_MODEL_PATH
                self.model = YOLO(str(local if local.exists() else "yolov8n-pose.pt"))
            except Exception:
                self.model = None

        maxlen = max(6, int(MAX_BUF_SEC * FPS_ASSUMED))
        self.cx_buf = deque(maxlen=maxlen)
        self.cy_buf = deque(maxlen=maxlen)
        self.area_buf = deque(maxlen=maxlen)
        self.last_stable_ts: Optional[float] = None

        self.captured_img: Optional[Image.Image] = None
        self.captured_ts: float = 0.0
        self.captured_metrics: List[str] = []

        self.latest_rgb: Optional[np.ndarray] = None
        self.last_metrics_lines: List[str] = []

        self.lock = threading.Lock()

    def _detect_bbox(self, rgb: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
        if not self.model:
            return None
        res = self.model.predict(rgb, imgsz=640, device="cpu", verbose=False)
        if len(res) == 0 or res[0].boxes is None:
            return None
        b = res[0].boxes.xyxy
        if b is None or len(b) == 0:
            return None
        b = b.cpu().numpy()
        areas = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
        i = int(np.argmax(areas))
        x1, y1, x2, y2 = b[i].astype(int).tolist()
        return x1, y1, x2, y2

    def _extract_keypoints(self, res) -> Dict[int, Tuple[float, float] | None]:
        kps: Dict[int, Tuple[float, float] | None] = {}
        try:
            if res and res[0].keypoints is not None and len(res[0].keypoints) > 0:
                xy = res[0].keypoints.xy[0].cpu().numpy()
                for i in range(xy.shape[0]):
                    kps[i] = (float(xy[i, 0]), float(xy[i, 1]))
        except Exception:
            pass
        return kps

    def _update_stillness(self, bbox, w, h) -> bool:
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        area = max(1.0, (x2 - x1) * (y2 - y1))
        rel_area = area / float(w * h)

        if rel_area < MIN_FACE_AREA:
            self.last_stable_ts = None
            return False

        self.cx_buf.append(cx)
        self.cy_buf.append(cy)
        self.area_buf.append(rel_area)

        need_len = int(STILLNESS_SEC * FPS_ASSUMED * 0.6)
        if len(self.cx_buf) < max(3, need_len):
            self.last_stable_ts = None
            return False

        stdx = float(np.std(self.cx_buf))
        stdy = float(np.std(self.cy_buf))
        stda = float(np.std(self.area_buf))
        stable_now = (
            stdx < MOTION_EPS_CXCY
            and stdy < MOTION_EPS_CXCY
            and stda < MOTION_EPS_AREA
        )

        now = time.time()
        if stable_now:
            if self.last_stable_ts is None:
                self.last_stable_ts = now
            return (now - self.last_stable_ts) >= STILLNESS_SEC
        else:
            self.last_stable_ts = None
            return False

    def _stamp_capture(self):
        self.captured_img = Image.fromarray(self.latest_rgb)
        self.captured_ts = time.time()
        self.captured_metrics = list(self.last_metrics_lines)
        self.last_stable_ts = None

    def capture_now(self) -> bool:
        if self.latest_rgb is None:
            return False
        with self.lock:
            self._stamp_capture()
        return True

    def recv(self, frame):
        import av

        img_bgr = frame.to_ndarray(format="bgr24")
        img_rgb = img_bgr[:, :, ::-1]
        h, w, _ = img_rgb.shape
        self.latest_rgb = img_rgb

        if self.model:
            res = self.model.predict(img_rgb, imgsz=640, device="cpu", verbose=False)
            plotted = res[0].plot()[:, :, ::-1]
            kps = self._extract_keypoints(res)
            lines = format_metrics(kps)
            with self.lock:
                self.last_metrics_lines = lines
            pil = Image.fromarray(plotted)
            pil = draw_tiny_metrics_top_right(pil, lines, size=16, margin=10)
            out_rgb = np.array(pil)
        else:
            with self.lock:
                self.last_metrics_lines = ["(pose model unavailable)"]
            out_rgb = img_rgb

        bbox = self._detect_bbox(img_rgb) if self.model else None
        if bbox and self._update_stillness(bbox, w, h):
            with self.lock:
                if time.time() - self.captured_ts > 0.35:
                    self._stamp_capture()

        out_bgr = out_rgb[:, :, ::-1]
        return av.VideoFrame.from_ndarray(out_bgr, format="bgr24")


# =================== PAGE LAYOUT ===================

st.set_page_config(page_title=APP_TITLE, layout="wide")

st.markdown(
    f"""
<style>
section[data-testid="stSidebar"] {{ display: none !important; }}
header, footer, [data-testid="stToolbar"] {{ visibility: hidden !important; }}

body {{
  background-color: #f5f5f7;
}}
.block-container {{
  padding-top: 0.6rem;
  padding-bottom: 0.6rem;
  max-width: 1700px;
}}

h1 {{
  font-family: -apple-system, BlinkMacSystemFont, "SF Pro Display", system-ui, sans-serif;
  letter-spacing: 0.04em;
  font-weight: 700;
}}

.left-col .cam-wrap {{
  position: relative;
  height: 80vh;
  width: 100%;
  overflow: hidden;
  border-radius: 18px;
  background: #111;
  box-shadow: 0 18px 45px rgba(0,0,0,0.3);
}}
.left-col .cam-wrap video {{
  height: 100% !important;
  width: auto !important;
  object-fit: cover !important;
  border-radius: 18px !important;
}}

.right-col .art-wrap {{
  position: relative;
  height: 80vh;
  max-width: {RIGHT_IMG_MAXW}px;
  overflow: hidden;
  margin: 0 auto;
  border-radius: 18px;
  background: #050505;
  box-shadow: 0 18px 45px rgba(0,0,0,0.35);
}}
.right-col .art-wrap img {{
  display: block;
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
}}

button[kind="secondary"] {{
  border-radius: 999px !important;
}}

</style>
""",
    unsafe_allow_html=True,
)

st.title(APP_TITLE)
st.caption(
    "Hold still for ~3–5 seconds to auto-capture, or press the button to capture manually. "
    "Left: live video with pose. Right: matched artwork with tiny pink pose metrics."
)

try:
    st.autorefresh(interval=700, key="ear_auto", limit=None)
except Exception:
    pass

left, right = st.columns([1, 1], gap="large")

if "countdown_target" not in st.session_state:
    st.session_state["countdown_target"] = None
if "last_match" not in st.session_state:
    st.session_state["last_match"] = None
if "last_metrics" not in st.session_state:
    st.session_state["last_metrics"] = []
if "last_ts" not in st.session_state:
    st.session_state["last_ts"] = 0.0
if "recent_files" not in st.session_state:
    st.session_state["recent_files"] = []  # type: List[str]

API_URL = DEFAULT_API_URL


# =================== LEFT PANEL ===================

with left:
    st.subheader("Live")
    st.markdown('<div class="left-col">', unsafe_allow_html=True)
    st.markdown('<div class="cam-wrap">', unsafe_allow_html=True)

    ctx = webrtc_streamer(
        key="ear-curatorial-final",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIGURATION,
        media_stream_constraints={"video": True, "audio": False},
        video_processor_factory=CuratorialProcessor,
        async_processing=True,
    )

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("<div style='height:10px'></div>", unsafe_allow_html=True)

    c1, c2 = st.columns(2)

    with c1:
        if st.button("📸 Capture (wait 3s)", use_container_width=True):
            st.session_state["countdown_target"] = time.time() + 3.0

    with c2:
        if st.button("⚡ Instant Capture", use_container_width=True):
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")

    if st.session_state["countdown_target"]:
        remain = st.session_state["countdown_target"] - time.time()
        if remain > 0:
            st.info(f"Capturing in {remain:.1f}s… Hold still.")
        else:
            if ctx and ctx.video_processor:
                ok = ctx.video_processor.capture_now()
                st.toast("Captured." if ok else "Try again.", icon="✅" if ok else "⚠️")
            st.session_state["countdown_target"] = None
            force_rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# =================== RIGHT PANEL ===================

with right:
    st.subheader("Matched artwork")

    # Match mode control
    match_mode = st.selectbox(
        "Match mode",
        ["default", "portrait_only", "high_value_only", "portrait_high_value"],
        index=0,
        help="Use 'portrait_only' to bias towards clear portraits; "
             "'high_value_only' for masterpieces.",
    )

    st.markdown('<div class="right-col">', unsafe_allow_html=True)
    st.markdown('<div class="art-wrap">', unsafe_allow_html=True)

    ph = st.empty()

    proc: Optional[CuratorialProcessor] = None
    if ctx and ctx.video_processor:
        proc = ctx.video_processor

    if not proc:
        ph.info("Initializing camera…")
    else:
        with proc.lock:
            cap_img = proc.captured_img
            cap_ts = getattr(proc, "captured_ts", 0.0)
            cap_metrics = list(getattr(proc, "captured_metrics", []))

        if cap_img is not None and cap_ts > st.session_state["last_ts"]:
            st.session_state["last_ts"] = cap_ts
            st.session_state["last_metrics"] = cap_metrics

            buf = io.BytesIO()
            cap_img.save(buf, format="JPEG")
            buf.seek(0)

            files = {"image": ("frame.jpg", buf.getvalue(), "image/jpeg")}
            # Ask backend for Top-10, we'll do diversity selection here
            data = {
                "museum": "mixed",
                "topk": TAIL_TOP_K,
                "mode": match_mode,
            }

            try:
                resp = requests.post(API_URL, files=files, data=data, timeout=30)
                resp.raise_for_status()
                st.session_state["last_match"] = resp.json()
            except Exception as exc:
                st.session_state["last_match"] = {"error": str(exc)}

            force_rerun()

        payload = st.session_state.get("last_match")

        if not payload:
            ph.info("Hold still or press capture to match…")
        elif "error" in payload:
            ph.error(f"Backend error: {payload['error']}")
        else:
            results = payload.get("results") or []
            if not results:
                ph.warning("No matches returned.")
            else:
                recent = st.session_state.get("recent_files", [])

                # ---------- diversity + mild randomness ----------
                # Decide whether to start looking from the tail (ranks 4–10)
                use_tail_first = (
                    random.random() < TAIL_RANDOM_PROB
                    and len(results) > PRIMARY_TOP_K
                )
                primary_slice = results[:PRIMARY_TOP_K]
                tail_slice = results[PRIMARY_TOP_K:TAIL_TOP_K]

                ordered_candidates = (
                    (tail_slice + primary_slice)
                    if use_tail_first
                    else (primary_slice + tail_slice)
                )

                chosen = None
                chosen_fname = None

                for r in ordered_candidates:
                    fname = (
                        r.get("filename")
                        or r.get("file")
                        or r.get("image_path")
                        or r.get("path")
                    )
                    if not fname:
                        continue
                    if fname not in recent:
                        chosen = r
                        chosen_fname = fname
                        break

                # Fallback: if all candidates are "recent", use strict Top-1
                if chosen is None:
                    chosen = results[0]
                    chosen_fname = (
                        chosen.get("filename")
                        or chosen.get("file")
                        or chosen.get("image_path")
                        or chosen.get("path")
                    )

                # Update recent memory
                if chosen_fname:
                    if chosen_fname in recent:
                        recent.remove(chosen_fname)
                    recent.insert(0, chosen_fname)
                    del recent[MAX_RECENT:]
                    st.session_state["recent_files"] = recent

                filename = chosen_fname

                img_path = ensure_image_path(filename or "")
                if not img_path:
                    ph.error(f"Image not found: {filename}")
                else:
                    painting = safe_open_image(img_path)
                    if painting is None:
                        ph.error(f"Failed to open: {img_path}")
                    else:
                        meta_row = lookup_meta(str(filename))
                        meta = {
                            "artist": meta_row.get("artist")
                            or chosen.get("artist")
                            or "artist name",
                            "year": meta_row.get("year") or chosen.get("year") or "",
                            "price_text": meta_row.get("price_text")
                            or meta_row.get("auction_price_usd")
                            or "",
                        }

                        painted = overlay_right_labels(painting, meta)
                        metrics = st.session_state.get("last_metrics") or []
                        painted = draw_tiny_metrics_top_right(
                            painted, metrics, size=16, margin=12
                        )

                        w = min(RIGHT_IMG_MAXW, painted.width)
                        caption = f"{meta_row.get('title','')} — {meta.get('artist','')}"
                        ph.image(painted, caption=caption, width=w)

    st.markdown("</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)


In [ ]:
"""
Fetch a small curated portrait dataset from The Met Museum Open Access API.

This script:
1. Searches for portraits / female portrait / male portrait
2. Downloads the first ~80 high-quality, CC0 images
3. Saves them into data/met/images/
4. Writes a metadata CSV: data/met/portrait_works.csv

You only need to run this once.

Usage:
    python scripts/fetch_met_portraits.py
"""

import csv
import os
from pathlib import Path
import requests
import time

ROOT = Path(__file__).resolve().parents[1]
OUT_DIR = ROOT / "data" / "met"
IMG_DIR = OUT_DIR / "images"
OUT_CSV = OUT_DIR / "portrait_works.csv"

SEARCH_TERMS = [
    "portrait",
    "woman portrait",
    "man portrait",
    "lady portrait",
    "self-portrait",
]

MAX_RESULTS = 80  # total images you want

# Ensure folders exist
OUT_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)


def met_search_ids(query: str):
    """Search Met API and return list of objectIDs."""
    url = "https://collectionapi.metmuseum.org/public/collection/v1/search"
    params = {
        "q": query,
        "hasImages": "true",
        "isOnView": "true"
    }
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return []
    data = r.json()
    ids = data.get("objectIDs") or []
    return ids[:100]  # get up to 100 per keyword


def fetch_object(object_id: int):
    """Get object details + primaryImage."""
    url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}"
    r = requests.get(url, timeout=10)
    if r.status_code != 200:
        return None
    data = r.json()

    img = data.get("primaryImage") or data.get("primaryImageSmall")
    if not img:
        return None

    # Only accept CC0
    if data.get("isPublicDomain") != True:
        return None

    title = data.get("title") or ""
    artist = data.get("artistDisplayName") or "Unknown"
    year = data.get("objectDate") or ""
    museum = "Metropolitan Museum of Art, New York"
    permalink = data.get("objectURL") or ""

    return {
        "object_id": object_id,
        "title": title,
        "artist": artist,
        "year": year,
        "image_url": img,
        "museum": museum,
        "permalink": permalink,
    }


def download_image(url: str, out_path: Path):
    """Save image to disk."""
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            return True
    except Exception:
        return False
    return False


def main():
    print("=== Fetching Met Museum portraits ===")

    all_ids = []
    for q in SEARCH_TERMS:
        ids = met_search_ids(q)
        print(f"{q} → {len(ids)} ids")
        all_ids.extend(ids)
        time.sleep(0.5)

    all_ids = list(dict.fromkeys(all_ids))  # remove duplicates
    print(f"Total unique IDs: {len(all_ids)}")

    rows = []
    count = 0

    for oid in all_ids:
        if count >= MAX_RESULTS:
            break

        try:
            item = fetch_object(oid)
        except Exception:
            item = None

        if not item:
            continue

        filename = f"{item['object_id']}.jpg"
        out_img = IMG_DIR / filename

        ok = download_image(item["image_url"], out_img)
        if not ok:
            continue

        row = {
            "filename": filename,
            "title": item["title"],
            "artist": item["artist"],
            "year": item["year"],
            "museum": item["museum"],
            "license": "CC0",
            "price_text": "",
            "auction_price_usd": "",
            "permalink": item["permalink"],
        }
        rows.append(row)
        count += 1
        print(f"[{count}] Saved {filename}")

        time.sleep(0.2)

    # Write metadata CSV
    with open(OUT_CSV, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "filename",
                "title",
                "artist",
                "year",
                "museum",
                "license",
                "price_text",
                "auction_price_usd",
                "permalink",
            ],
        )
        writer.writeheader()
        writer.writerows(rows)

    print(f"\nDone! Saved {count} images.")
    print(f"Images → {IMG_DIR}")
    print(f"CSV    → {OUT_CSV}")


if __name__ == "__main__":
    main()
